In [ ]:
# In-built packages/libraries

import numpy as np
import pandas as pd
import re
import os
from PIL import Image
import anndata
from anndata import AnnData
import base64
import mimetypes
from pathlib import Path
from random import sample
import matplotlib.pyplot as plt
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.models import DataTable, TableColumn, ColumnDataSource, BoxSelectTool, PanTool, LassoSelectTool
from bokeh.models import Div, WheelZoomTool, CustomJS, Slider, Spinner, Label, PointDrawTool, Dropdown
from bokeh.models import LabelSet, Range1d, Arrow, OpenHead, Button, Toggle, SaveTool, Select, CDSView, BooleanFilter
from bokeh.models.layouts import TabPanel, Tabs
from bokeh.layouts import column, row
from bokeh.events import ButtonClick, Pan, MouseEnter, RangesUpdate, ValueSubmit, Tap, MouseMove
from bokeh.plotting import figure, show

from bokeh.models import Legend, LegendItem
from bokeh.transform import factor_cmap, factor_mark

from bokeh.palettes import Category20
from bokeh.palettes import mpl

from bokeh.io import push_notebook, output_notebook
output_notebook()
%matplotlib inline

In [ ]:
def generate_colors_hex(num_colors):
    """
    Generate a list of distinct high-contrast colors in hexadecimal format.
    """
    base_colors = plt.cm.tab10(np.linspace(0, 1, 10))  # Using tab10 colormap for base colors
    colors_rgba = base_colors[np.arange(num_colors) % len(base_colors)]
    np.random.shuffle(colors_rgba)
    
    # Convert RGBA to hexadecimal
    colors_hex = [f"#{rgb_to_hex(color)}" for color in (colors_rgba[:, :3] * 255).astype(int)]
    return colors_hex

def rgb_to_hex(rgb):
    """
    Convert RGB color values to hexadecimal.
    """
    return '%02x%02x%02x' % tuple(rgb)

In [ ]:
# for organizing input spatio-molecular data 

def dictionary_for_data(filename, x_coordinates, y_coordinates, cell_ids, scale_for_sm_data, scale_bar, number_of_data_points_to_be_displayed, clusters='none'):

    name, extension = os.path.splitext(filename)

    if extension == '.csv' or extension == '.xlsx':
        if extension == '.csv':
            DATA = pd.read_csv(filename) 
        if extension == '.xlsx':
            DATA = pd.read_excel(filename) 
        
        if clusters == 'none':
            clusters = [np.nan] * len(DATA[x_coordinates])
            DATA['Clusters'] = clusters
            clusters = 'Clusters'
            clus_vals = {}
            clus_vals['nan'] = len(list(DATA['Clusters']))
            clus_vals_indices_selected = {}
            clus_vals_indices_before = {}
            clus_vals_indices_discarded = {}
            clus_vals_indices_before['nan'] = [i for i in range(0, len(list(DATA['Clusters'])))]
        
        else:
            clus_vals = {}
            clus_vals_indices_selected = {}
            clus_vals_indices_before = {}
            clus_vals_indices_discarded = {}

            for index, i in enumerate(list(DATA[clusters])):

                if i not in clus_vals.keys():
                    clus_vals[i] = 1
                    clus_vals_indices_before[i] = []
                    clus_vals_indices_before[i].append(index)
                else:
                    clus_vals[i] += 1
                    clus_vals_indices_before[i].append(index)
        
        subsampled_df = pd.DataFrame()
        remaining_df = pd.DataFrame()

        subset_of_data_for_cds_view_index = []
        subset_of_data_for_cds_view_value = []
        
        if number_of_data_points_to_be_displayed == 'all':
            number_of_data_points_to_be_displayed = len(DATA[x_coordinates])
        
        for i in clus_vals.keys():
            
            fraction_of_this_cluster_in_the_actual_dataset = clus_vals[i] / len(DATA[x_coordinates])
            
            downsampled_amount = fraction_of_this_cluster_in_the_actual_dataset * number_of_data_points_to_be_displayed
            
            clus_vals_indices_selected[i] = sample(clus_vals_indices_before[i], int(downsampled_amount))
            #clus_vals_indices_discarded[i] = [value for value in clus_vals_indices_before[i] if value not in clus_vals_indices_selected[i]]

            for j in clus_vals_indices_selected[i]:
                subsampled_df = subsampled_df.append(DATA.iloc[j])

            for z in clus_vals_indices_before[i]:
                if z in clus_vals_indices_selected[i]:
                    subset_of_data_for_cds_view_index.append(z)
                    subset_of_data_for_cds_view_value.append(True)
                else:
                    subset_of_data_for_cds_view_index.append(z)
                    subset_of_data_for_cds_view_value.append(False)
                    
        # remaining_df will have ALL the coordinates 
        
        remaining_df = DATA
        in_cdsview = []

        for df_index, df_value in enumerate(remaining_df.index.to_list()):     
            in_cdsview.append(subset_of_data_for_cds_view_value[subset_of_data_for_cds_view_index.index(df_index)])
        
        remaining_df['in_cdsview'] = in_cdsview

        LIST_dict_of_data = [] 
        LIST_list_of_keys = [] 
        LIST_max_x_data = [] 
        LIST_max_y_data = [] 
        LIST_avg_xcoord = [] 
        LIST_avg_ycoord = [] 
        
        actual_number_datapoints_displayed_on_the_plot = len(subsampled_df)
        total_number_of_datapoints_in_the_smdata_file = len(remaining_df)
        
        for index_of_dfs, data in enumerate([subsampled_df, remaining_df]):
            
            data = data.reset_index()
            data.drop(['index'], axis=1, inplace=True)
            
            c = data[clusters].to_list()

            data[x_coordinates] = (data[x_coordinates] / scale_for_sm_data) * scale_bar
            data[y_coordinates] = (data[y_coordinates] / scale_for_sm_data) * scale_bar 

            max_x_data = max(data[x_coordinates])
            max_y_data = max(data[y_coordinates])

            avg_xcoord=[]
            avg_ycoord=[]
            avg_xcoord.append(sum(data[x_coordinates])/len(data[x_coordinates]))
            avg_ycoord.append(sum(data[y_coordinates])/len(data[y_coordinates]))

            total_clus = list()

            if clusters == 'Clusters':
                total_clus.append(c[0])
            
            else:
                for i in c:
                    if i not in total_clus:
                        total_clus.append(i)

            dict_of_data = {}
            for i in total_clus:
                if type(i) == int or type(i) == float:
                    dict_of_data["C_{}".format(i)] = {}
                    dict_of_data["C_{}".format(i)]['x']=[]
                    dict_of_data["C_{}".format(i)]['y']=[]
                    dict_of_data["C_{}".format(i)]['CellIDs']=[]
                    dict_of_data["C_{}".format(i)]['clusters']=[]
                    if index_of_dfs != 0:
                        dict_of_data["C_{}".format(i)]['in_cdsview']=[]
                if type(i) == str:
                    dict_of_data[i] = {}
                    dict_of_data[i]['x']=[]
                    dict_of_data[i]['y']=[]
                    dict_of_data[i]['CellIDs']=[]
                    dict_of_data[i]['clusters']=[]
                    if index_of_dfs != 0:
                        dict_of_data[i]['in_cdsview']=[]

            neg_count = len(list(filter(lambda x: (x < 0), data[x_coordinates])))
            pos_count = len(data[x_coordinates]) - neg_count

            if pos_count >= neg_count:
                for index, value in enumerate(data[clusters]):
                    if type(value) == int or type(value) == float:
                        for key in dict_of_data:
                            if "C_{}".format(value) == key:
                                dict_of_data[key]['x'].append(data[x_coordinates][index])
                                dict_of_data[key]['y'].append(data[y_coordinates][index])
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])
                                
                    if type(value) == str:
                        for key in dict_of_data:
                            if value == key:
                                dict_of_data[key]['x'].append(data[x_coordinates][index])
                                dict_of_data[key]['y'].append(data[y_coordinates][index])
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])

            if neg_count > pos_count:
                for index, value in enumerate(data[clusters]):
                    if type(value) == int or type(value) == float:
                        for key in dict_of_data:
                            if "C_{}".format(value) == key:
                                dict_of_data[key]['x'].append((data[x_coordinates][index]-min(data[x_coordinates])))
                                dict_of_data[key]['y'].append((data[y_coordinates][index]-min(data[y_coordinates])))
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])
                    if type(value) == str:
                        for key in dict_of_data:
                            if value == key:
                                dict_of_data[key]['x'].append((data[x_coordinates][index]-min(data[x_coordinates])))
                                dict_of_data[key]['y'].append((data[y_coordinates][index]-min(data[y_coordinates])))
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])
                                
            list_of_keys = []
            for key in dict_of_data.keys():
                list_of_keys.append(key)
            
            LIST_dict_of_data.append(dict_of_data)
            LIST_list_of_keys.append(list_of_keys)
            LIST_max_x_data.append(max_x_data) 
            LIST_max_y_data.append(max_y_data) 
            LIST_avg_xcoord.append(avg_xcoord) 
            LIST_avg_ycoord.append(avg_ycoord) 

    if extension == '.h5ad':
        DATA = anndata.read_h5ad(filename)   
        
        if clusters == 'none':
            clusters = [np.nan] * len(DATA.obs[x_coordinates])
            DATA.obs['Clusters'] = clusters
            clusters = 'Clusters'
            clus_vals = {}
            clus_vals['nan'] = len(list(DATA.obs['Clusters']))
            clus_vals_indices_selected = {}
            clus_vals_indices_before = {}
            clus_vals_indices_discarded = {}
            clus_vals_indices_before['nan'] = [i for i in range(0, len(list(DATA.obs['Clusters'])))]
        
        else:
            clus_vals = {}
            clus_vals_indices_selected = {}
            clus_vals_indices_before = {}
            clus_vals_indices_discarded = {}

            for index, i in enumerate(list(DATA.obs[clusters])):

                if i not in clus_vals.keys():
                    clus_vals[i] = 1
                    clus_vals_indices_before[i] = []
                    clus_vals_indices_before[i].append(index)
                else:
                    clus_vals[i] += 1
                    clus_vals_indices_before[i].append(index)
        
        subsampled_df = pd.DataFrame()
        remaining_df = pd.DataFrame()

        subset_of_data_for_cds_view_index = []
        subset_of_data_for_cds_view_value = []
        
        if number_of_data_points_to_be_displayed == 'all':
            number_of_data_points_to_be_displayed = len(DATA.obs[x_coordinates])

        for i in clus_vals.keys():
                                                                                
            fraction_of_this_cluster_in_the_actual_dataset = clus_vals[i] / len(DATA.obs[x_coordinates])
            
            downsampled_amount = fraction_of_this_cluster_in_the_actual_dataset * number_of_data_points_to_be_displayed
            
            clus_vals_indices_selected[i] = sample(clus_vals_indices_before[i], int(downsampled_amount))
            #clus_vals_indices_discarded[i] = [value for value in clus_vals_indices_before[i] if value not in clus_vals_indices_selected[i]]

            for j in clus_vals_indices_selected[i]:
                subsampled_df = subsampled_df.append(DATA.obs.iloc[j])

            for z in clus_vals_indices_before[i]:
                if z in clus_vals_indices_selected[i]:
                    subset_of_data_for_cds_view_index.append(z)
                    subset_of_data_for_cds_view_value.append(True)
                else:
                    subset_of_data_for_cds_view_index.append(z)
                    subset_of_data_for_cds_view_value.append(False)
                    
        # remaining_df will have ALL the coordinates 
        
        remaining_df = DATA.obs
        in_cdsview = []

        for df_index, df_value in enumerate(remaining_df.index.to_list()):     
            in_cdsview.append(subset_of_data_for_cds_view_value[subset_of_data_for_cds_view_index.index(df_index)])
        
        remaining_df['in_cdsview'] = in_cdsview
        
        LIST_dict_of_data = [] 
        LIST_list_of_keys = [] 
        LIST_max_x_data = [] 
        LIST_max_y_data = [] 
        LIST_avg_xcoord = [] 
        LIST_avg_ycoord = [] 
        
        actual_number_datapoints_displayed_on_the_plot = len(subsampled_df)
        total_number_of_datapoints_in_the_smdata_file = len(remaining_df)
        
        for index_of_dfs, data in enumerate([subsampled_df, remaining_df]):
            
            data = data.reset_index()
            data.drop(['index'], axis=1, inplace=True)
            
            c = data[clusters].to_list()

            data[x_coordinates] = (data[x_coordinates] / scale_for_sm_data) * scale_bar
            data[y_coordinates] = (data[y_coordinates] / scale_for_sm_data) * scale_bar 

            max_x_data = max(data[x_coordinates])
            max_y_data = max(data[y_coordinates])

            avg_xcoord=[]
            avg_ycoord=[]
            avg_xcoord.append(sum(data[x_coordinates])/len(data[x_coordinates]))
            avg_ycoord.append(sum(data[y_coordinates])/len(data[y_coordinates]))

            total_clus = list()

            if clusters == 'Clusters':
                total_clus.append(c[0])
            
            else:
                for i in c:
                    if i not in total_clus:
                        total_clus.append(i)

            dict_of_data = {}
            for i in total_clus:
                if type(i) == int or type(i) == float:
                    dict_of_data["C_{}".format(i)] = {}
                    dict_of_data["C_{}".format(i)]['x']=[]
                    dict_of_data["C_{}".format(i)]['y']=[]
                    dict_of_data["C_{}".format(i)]['CellIDs']=[]
                    dict_of_data["C_{}".format(i)]['clusters']=[]
                    if index_of_dfs != 0:
                        dict_of_data["C_{}".format(i)]['in_cdsview']=[]
                if type(i) == str:
                    dict_of_data[i] = {}
                    dict_of_data[i]['x']=[]
                    dict_of_data[i]['y']=[]
                    dict_of_data[i]['CellIDs']=[]
                    dict_of_data[i]['clusters']=[]
                    if index_of_dfs != 0:
                        dict_of_data[i]['in_cdsview']=[]

            neg_count = len(list(filter(lambda x: (x < 0), data[x_coordinates])))
            pos_count = len(data[x_coordinates]) - neg_count

            if pos_count >= neg_count:
                for index, value in enumerate(data[clusters]):
                    if type(value) == int or type(value) == float:
                        for key in dict_of_data:
                            if "C_{}".format(value) == key:
                                dict_of_data[key]['x'].append(data[x_coordinates][index])
                                dict_of_data[key]['y'].append(data[y_coordinates][index])
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])
                                
                    if type(value) == str:
                        for key in dict_of_data:
                            if value == key:
                                dict_of_data[key]['x'].append(data[x_coordinates][index])
                                dict_of_data[key]['y'].append(data[y_coordinates][index])
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])

            if neg_count > pos_count:
                for index, value in enumerate(data[clusters]):
                    if type(value) == int or type(value) == float:
                        for key in dict_of_data:
                            if "C_{}".format(value) == key:
                                dict_of_data[key]['x'].append((data[x_coordinates][index]-min(data[x_coordinates])))
                                dict_of_data[key]['y'].append((data[y_coordinates][index]-min(data[y_coordinates])))
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])
                    if type(value) == str:
                        for key in dict_of_data:
                            if value == key:
                                dict_of_data[key]['x'].append((data[x_coordinates][index]-min(data[x_coordinates])))
                                dict_of_data[key]['y'].append((data[y_coordinates][index]-min(data[y_coordinates])))
                                dict_of_data[key]['CellIDs'].append(data[cell_ids][index])
                                dict_of_data[key]['clusters'].append(key)
                                if index_of_dfs != 0:
                                    dict_of_data[key]['in_cdsview'].append(data['in_cdsview'][index])
                                
            list_of_keys = []
            for key in dict_of_data.keys():
                list_of_keys.append(key)
            
            LIST_dict_of_data.append(dict_of_data)
            LIST_list_of_keys.append(list_of_keys)
            LIST_max_x_data.append(max_x_data) 
            LIST_max_y_data.append(max_y_data) 
            LIST_avg_xcoord.append(avg_xcoord) 
            LIST_avg_ycoord.append(avg_ycoord) 

    return LIST_dict_of_data, LIST_list_of_keys, LIST_max_x_data, LIST_max_y_data, LIST_avg_xcoord, LIST_avg_ycoord, actual_number_datapoints_displayed_on_the_plot, total_number_of_datapoints_in_the_smdata_file

In [ ]:
def creating_required_number_of_input_dictionaries(file_names, x_coordinate_column_names,
                                                  y_coordinate_column_names, clusters_column_names, cell_id_column_names,
                                                  scale_for_sm_data, scale_bar, total_tabs, number_of_data_points_to_be_displayed):

    list_for_all_data_types = []
    list_for_storing_total_clusters_info_in_each_file = []
    list_for_storing_max_x_coordinate_of_each_file = []
    list_for_storing_max_y_coordinate_of_each_file = []
    list_for_storing_average_of_x_coordinated_of_each_file = []
    list_for_storing_average_of_y_coordinated_of_each_file = []
    
    list_for_all_data_types_remaining = []
    list_for_storing_total_clusters_info_in_each_file_remaining = []
    list_for_storing_max_x_coordinate_of_each_file_remaining = []
    list_for_storing_max_y_coordinate_of_each_file_remaining = []
    list_for_storing_average_of_x_coordinated_of_each_file_remaining = []
    list_for_storing_average_of_y_coordinated_of_each_file_remaining = []
    
    for i in range(len(total_tabs)):
        dictionary_for_each_DATA, total_clus, max_x_data, max_y_data, avg_xcoord, avg_ycoord, actual_number_datapoints_displayed_on_the_plot, total_number_of_datapoints_in_the_smdata_file = dictionary_for_data(file_names[i],
                                                                                                              x_coordinate_column_names[i],
                                                                                                              y_coordinate_column_names[i],
                                                                                                              cell_id_column_names[i],
                                                                                                              scale_for_sm_data[i], scale_bar[i], number_of_data_points_to_be_displayed[i],
                                                                                                              clusters_column_names[i])

        list_for_all_data_types.append(dictionary_for_each_DATA[0])
        list_for_storing_total_clusters_info_in_each_file.append(total_clus[0])
        list_for_storing_max_x_coordinate_of_each_file.append(max_x_data[0])
        list_for_storing_max_y_coordinate_of_each_file.append(max_y_data[0])
        list_for_storing_average_of_x_coordinated_of_each_file.append(avg_xcoord[0])
        list_for_storing_average_of_y_coordinated_of_each_file.append(avg_ycoord[0])
        
        list_for_all_data_types_remaining.append(dictionary_for_each_DATA[1])
        list_for_storing_total_clusters_info_in_each_file_remaining.append(total_clus[1])
        list_for_storing_max_x_coordinate_of_each_file_remaining.append(max_x_data[1])
        list_for_storing_max_y_coordinate_of_each_file_remaining.append(max_y_data[1])
        list_for_storing_average_of_x_coordinated_of_each_file_remaining.append(avg_xcoord[1])
        list_for_storing_average_of_y_coordinated_of_each_file_remaining.append(avg_ycoord[1])

    return list_for_all_data_types, list_for_all_data_types_remaining, list_for_storing_total_clusters_info_in_each_file, list_for_storing_total_clusters_info_in_each_file_remaining, list_for_storing_max_x_coordinate_of_each_file, list_for_storing_max_x_coordinate_of_each_file_remaining, list_for_storing_max_y_coordinate_of_each_file, list_for_storing_max_y_coordinate_of_each_file_remaining, list_for_storing_average_of_x_coordinated_of_each_file, list_for_storing_average_of_x_coordinated_of_each_file_remaining, list_for_storing_average_of_y_coordinated_of_each_file, list_for_storing_average_of_y_coordinated_of_each_file_remaining, actual_number_datapoints_displayed_on_the_plot, total_number_of_datapoints_in_the_smdata_file 

In [ ]:
# work space for boremi

def boremi(total_tabs, list_for_all_data_types, 
                      list_for_all_data_types_remaining, list_for_storing_total_clusters_info_in_each_file, 
                      list_for_storing_total_clusters_info_in_each_file_remaining, list_for_storing_max_x_coordinate_of_each_file, 
                      list_for_storing_max_x_coordinate_of_each_file_remaining, list_for_storing_max_y_coordinate_of_each_file, 
                      list_for_storing_max_y_coordinate_of_each_file_remaining, list_for_storing_average_of_x_coordinated_of_each_file, 
                      list_for_storing_average_of_x_coordinated_of_each_file_remaining, 
                      list_for_storing_average_of_y_coordinated_of_each_file, list_for_storing_average_of_y_coordinated_of_each_file_remaining,
                      imgs, title, scale_for_he_image, scale_for_sm_data, scale_bar, actual_number_datapoints_displayed_on_the_plot,
                      total_number_of_datapoints_in_the_smdata_file):

    tabs = []

    list_of_linear_buttons = []
    list_of_non_linear_buttons = []

    dict_for_toggle_vert = {}
    dict_for_toggle_vert["toggle_buttons"] = []

    dict_for_toggle_hori = {}
    dict_for_toggle_hori["toggle_buttons"] = []

    dict_for_image_save_button = {}
    dict_for_image_save_button["save_buttons"] = []
    
    dict_for_point_size = {}
    dict_for_point_size["sliders"] = []
    dict_for_point_size["spinners"] = []

    dict_for_point_size_non_linear = {}
    dict_for_point_size_non_linear["spinners"] = []

    dict_for_data_save_button_non_linear = {}
    dict_for_data_save_button_non_linear["save_buttons"] = []
    
    dict_for_data_pixel_size = {}
    dict_for_data_pixel_size["sliders"] = []
    dict_for_data_pixel_size["spinners"] = []

    dict_for_he_pixel_size = {}
    dict_for_he_pixel_size["sliders"] = []
    dict_for_he_pixel_size["spinners"] = []

    dict_for_he_rot_angle = {}
    dict_for_he_rot_angle["sliders"] = []
    dict_for_he_rot_angle["spinners"] = []
    
    dict_undo_button = {}
    dict_undo_button['buttons'] = []
    
    dict_redo_button = {}
    dict_redo_button['buttons'] = []
    
    dict_for_data_translocation_X = {}
    dict_for_data_translocation_Y = {}
    
    dict_for_data_translocation_X["sliders"] = []
    dict_for_data_translocation_Y["sliders"] = []
    
    dict_for_data_translocation_X["spinners"] = []
    dict_for_data_translocation_Y["spinners"] = []

    dict_for_img_number = {}
    dict_for_img_number["sliders"] = []
    dict_for_img_number["spinners"] = []

    dict_for_data_rot_angle = {}
    dict_for_data_rot_angle["sliders"] = []
    dict_for_data_rot_angle["spinners"] = []

    dict_for_data_save_button = {}
    dict_for_data_save_button["save_buttons"] = []

    dict_of_coordinates_for_source_final = {}
    dict_of_coordinates_for_source_final["xcoord"] = {}
    dict_of_coordinates_for_source_final["ycoord"] = {}

    dict_of_factors_for_data_resize = {}
    dict_of_factors_for_data_resize["numerators"] = {}
    dict_of_factors_for_data_resize["denominators"] = {}

    dict_for_scale_bar_pixel_size = {}
    dict_for_scale_bar_pixel_size["sliders"] = []
    dict_for_scale_bar_pixel_size["spinners"] = []
    
    dict_for_threshold = {}
    dict_for_threshold["spinners"] = []

    dict_for_active_array = {}
    dict_for_renderers = {}
    dict_of_sources = {}

    dict_of_sources_non_linear = {}
    dict_for_renderers_non_linear = {}
    items_for_legend = {}
    
    list_for_columndatasource_of_data_resize = []
    list_for_rotation_values = []
    list_for_data_table = []
    list_of_source_final = []
    spinners_for_all_data_point_size = []
    spinners_for_all = []
    
    spinners_for_all_data_point_size_non_linear = []
    spinners_for_all_non_linear = []
    
    list_for_rotation_values_remaining = []    
    list_for_translocation_values = []
    remaining_data_points_list_regardless_of_clusters = []
    
    div_cluster_heading = []
    div_undo_redo = []
    div_preview_imgs = []
    div_table_heading = []
    div_orientation = []
    div_translocation = []
    div_px_size = []
    div_rot_angle = []
    div_transform_heading = []
    div_download_heading=[]

    list_of_non_linear_buttons = []
    list_of_linear_buttons = []

    div4space1 = [] 
    div4space2 = []
    div4space3 = [] 
    div4space4 = []
    div4space5 = []
    div4space6 = []
    div4space7 = []
    
    undo_dict = {}
    redo_dict = {}
    dict_for_widget_names_function_names = {}
    identifier = {}

    log_file = {}

    colors = []
    uris = []
    divs_of_imgs = []
    dummy_dict = {}

    combined_data = []
    combined_source = []
    legend_plot = []
    draw_tool = []

    for i in imgs:
        file_extension = os.path.splitext(i)[1]
        mimetypes.init()
        if file_extension in mimetypes.types_map:
            with open(i, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
            encoded_string = 'data:' + mimetypes.types_map[file_extension] + ';base64,' + encoded_string

        else:
            raise ValueError(f'Unknown file extension {file_extension!r} of file {i!r}!')

        image = Image.open(i)
        width, height = image.size
        aspectratio = width/height
        uris.append(encoded_string)
        divs_of_imgs.append(Div(text = r'<img src= ' + encoded_string + r' style="width:100px;aspect_ratio:{};opacity:1" ></img>'. 
                               format(aspectratio), visible = True))

    for number_of_tab in range(len(total_tabs)):
        
        log_file[number_of_tab] = ColumnDataSource(data=dict(method=[], value=[]))
        
        dict_of_factors_for_data_resize["numerators"][number_of_tab]=[]
        dict_of_factors_for_data_resize["numerators"][number_of_tab].append(scale_for_sm_data[number_of_tab])
        dict_of_factors_for_data_resize["denominators"][number_of_tab]=[]
        dict_of_factors_for_data_resize["denominators"][number_of_tab].append(1)
        list_for_columndatasource_of_data_resize.append(ColumnDataSource(data=dict(x=dict_of_factors_for_data_resize["numerators"][number_of_tab], 
                                                                y=dict_of_factors_for_data_resize["denominators"][number_of_tab])))

        list_for_rotation_values.append(ColumnDataSource(data=dict(x=list_for_storing_average_of_x_coordinated_of_each_file[number_of_tab], 
                                                     y=list_for_storing_average_of_y_coordinated_of_each_file[number_of_tab], 
                                                     rotval=[0])))
        
        list_for_rotation_values_remaining.append(ColumnDataSource(data=dict(x=list_for_storing_average_of_x_coordinated_of_each_file_remaining[number_of_tab], 
                                                     y=list_for_storing_average_of_y_coordinated_of_each_file_remaining[number_of_tab], 
                                                     rotval=[0])))
        
        list_for_translocation_values.append(ColumnDataSource(data=dict(x=[0], y=[0])))

        dummy_dict[number_of_tab] = {}
        dummy_dict[number_of_tab]['x'] = []
        dummy_dict[number_of_tab]['y'] = []
        dummy_dict[number_of_tab]['CellIDs'] = []
        for key, value in list_for_all_data_types_remaining[number_of_tab].items():
            dummy_dict[number_of_tab]['x'] = dummy_dict[number_of_tab]['x'] + value['x']
            dummy_dict[number_of_tab]['y'] = dummy_dict[number_of_tab]['y'] + value['y']
            dummy_dict[number_of_tab]['CellIDs'] = dummy_dict[number_of_tab]['CellIDs'] + value['CellIDs']
        
        #list_for_all_data_types_remaining[number_of_tab] = dummy_dict
            
        #remaining_data_points_list_regardless_of_clusters.append(ColumnDataSource(data=dict(x=list_for_all_data_types_remaining[number_of_tab]['x'], 
        #                                             y=list_for_all_data_types_remaining[number_of_tab]['y'], 
        #                                             cellid=list_for_all_data_types_remaining[number_of_tab]['CellIDs'])))
        
        remaining_data_points_list_regardless_of_clusters.append(ColumnDataSource(data=dict(x=dummy_dict[number_of_tab]['x'], 
                                                     y=dummy_dict[number_of_tab]['y'], 
                                                     cellid=dummy_dict[number_of_tab]['CellIDs'])))
            
        num=[]       
        den=[]       
        for i in range(len(imgs)):
            num.append(scale_for_he_image)
            den.append(1)
        division_factor_for_resize_he = ColumnDataSource(data=dict(x=num, y=den))

        division_factor_for_resize_scale_bar = ColumnDataSource(data=dict(x=[1], y=[scale_bar[number_of_tab]]))

        width_of_images=[]
        height_of_images=[]
        for i in imgs:
            image = Image.open(i)
            width, height = image.size
            width_of_images.append((width/scale_for_he_image) * scale_bar[number_of_tab])
            height_of_images.append((height/scale_for_he_image) * scale_bar[number_of_tab])

        xmax_overall = max(width_of_images)
        ymax_overall = max(height_of_images)

        if number_of_tab == 0:
            visibility = True
        else:
            visibility = False

        df = pd.DataFrame(uris, columns=['imgs'])
        source_img = ColumnDataSource(df)
        
        df_2 = pd.DataFrame(imgs, columns=['imgs'])
        source_img_2 = ColumnDataSource(df_2)

        max_value = max(list_for_storing_max_x_coordinate_of_each_file[number_of_tab], 
                        list_for_storing_max_y_coordinate_of_each_file[number_of_tab], 
                        xmax_overall, ymax_overall)
        p = figure(width=1200, height=1300, x_range=(0, max_value), y_range=(0, max_value), tools=[], title=f'BoReMi' + '\n'+ f'Number of cells displayed on the plot: {actual_number_datapoints_displayed_on_the_plot} out of a total {total_number_of_datapoints_in_the_smdata_file} cells')
        
        im = p.image_url(url=[source_img.data['imgs'][0]], 
                         x=width_of_images[0]/2, 
                         y=height_of_images[0]/2, 
                         w=width_of_images[0], 
                         h=height_of_images[0], 
                         anchor='center', angle_units='rad', angle=0)
        p.add_tools(WheelZoomTool(), PanTool(dimensions="both"), BoxSelectTool(), SaveTool())
        
        # render_mode='css'
        my_arrow = Arrow(end=OpenHead(size=10), start=OpenHead(size=10), line_color="black", visible=True,
                         x_start=max_value-1500, y_start=600, x_end=max_value-500, y_end=600, line_alpha=1,line_width=2)
        p.add_layout(my_arrow)

        labels = Label(x=max_value-1500, y=600, x_offset=15, y_offset=-30, x_units='data', y_units='data', 
                       text='1 mm', border_line_color='black', border_line_alpha=1.0,
                       background_fill_color='white', background_fill_alpha=1.0,
                       text_font_size='10px', visible=True)

        p.add_layout(labels)
        
        dict_of_sources[number_of_tab] = {}
        dict_for_renderers[number_of_tab] = []

        ordered = pd.DataFrame()
        clusters_ordered = []
        ascii_ = []
        if type(list_for_storing_total_clusters_info_in_each_file[number_of_tab][0]) == str:
            for i in list_for_storing_total_clusters_info_in_each_file[number_of_tab]:
                sum_ascii = 0
                for j in i:
                    sum_ascii = sum_ascii + ord(j)
                ascii_.append(sum_ascii)
                clusters_ordered.append(i)
            ordered['Clusters']=clusters_ordered
            ordered['ASCII values']=ascii_
            ordered = ordered.sort_values('ASCII values')
            ordered.head(40)
            list_for_storing_total_clusters_info_in_each_file[number_of_tab] = ordered['Clusters'].to_list()

        if type(list_for_storing_total_clusters_info_in_each_file[number_of_tab][0]) == int or type(list_for_storing_total_clusters_info_in_each_file[number_of_tab][0]) == float:
            list_for_storing_total_clusters_info_in_each_file[number_of_tab].sort()

        
        # for modifying data point size
        spinners_for_all_data_point_size.append(Spinner(title="Data Point Size (lowest: 0.06, highest: 10):", 
                          low=0.05, high=10, step=0.5, value=1, width=200, visible=visibility))

        spinners_for_all_data_point_size_non_linear.append(Spinner(title="Data Point Size (lowest: 0.06, highest: 10):", 
                          low=0.05, high=10, step=0.5, value=1, width=200, visible=False))

        colors.append(generate_colors_hex(len(list_for_storing_total_clusters_info_in_each_file[number_of_tab])))

        items_for_legend[number_of_tab] = []

        legend_plot.append(figure(height=200, width=1000, toolbar_location=None))
        legend_plot[number_of_tab].axis.visible = False
        legend_plot[number_of_tab].grid.visible = False
        legend_plot[number_of_tab].outline_line_color = None

        for index, i in enumerate(list_for_storing_total_clusters_info_in_each_file[number_of_tab]):
            if type(i) == int or type(i) == float:
                dict_of_sources[number_of_tab]["C{}".format(i)] = ColumnDataSource(data=dict(x=list_for_all_data_types[number_of_tab]["C_{}".format(i)]['x'], 
                                                                      y=list_for_all_data_types[number_of_tab]["C_{}".format(i)]['y'],
                                                                      locked=[False] * len(list_for_all_data_types[number_of_tab]["C_{}".format(i)]['y']),
                                                              original_x=list_for_all_data_types[number_of_tab]["C_{}".format(i)]['x'].copy(), 
                                                               original_y=list_for_all_data_types[number_of_tab]["C_{}".format(i)]['y'].copy(),
                                                                click_count=[0] * len(list_for_all_data_types[number_of_tab]["C_{}".format(i)]['y'])))

                R = p.scatter(x='x', y='y', source=dict_of_sources[number_of_tab]["C{}".format(i)], color=colors[number_of_tab][index], size=1)

                dict_for_renderers[number_of_tab].append(R)

                L = legend_plot[number_of_tab].scatter([0], [0], size=0, color=colors[number_of_tab][index])
                items_for_legend[number_of_tab].append(LegendItem(label="C_{}".format(i), renderers=[R, L]))


            if type(i) == str:
                dict_of_sources[number_of_tab][i] = ColumnDataSource(data=dict(x=list_for_all_data_types[number_of_tab][i]['x'], 
                                                        y=list_for_all_data_types[number_of_tab][i]['y'],
                                                        locked=[False] * len(list_for_all_data_types[number_of_tab][i]['y']),
                                                        original_x=list_for_all_data_types[number_of_tab][i]['x'].copy(), 
                                                        original_y=list_for_all_data_types[number_of_tab][i]['y'].copy(),
                                                        click_count=[0] * len(list_for_all_data_types[number_of_tab][i]['y'])))

                R = p.scatter(x='x', y='y', source=dict_of_sources[number_of_tab][i], color=colors[number_of_tab][index], size=1)

                dict_for_renderers[number_of_tab].append(R)

                L = legend_plot[number_of_tab].scatter([0], [0], size=0, color=colors[number_of_tab][index])
                items_for_legend[number_of_tab].append(LegendItem(label=i, renderers=[R, L]))

        legend_x_position = 0
        legend_y_position = 0
        subtract_from_y = 2
        
        #legend_linear=[]

        for ITEM in range(0, len(items_for_legend[number_of_tab]), 7):
            LEGEND = Legend(items=items_for_legend[number_of_tab][ITEM:ITEM+7],
                      location=(legend_x_position, legend_y_position))
            #p.add_layout(legend_linear[-1], 'below')
            legend_x_position = legend_x_position + 120

            legend_plot[number_of_tab].add_layout(LEGEND, 'center')
            LEGEND.label_text_font_size = '10pt'
            LEGEND.click_policy = "hide"
            #LEGEND.visible = True

        legend_plot[number_of_tab].visible = visibility

        #legend_x_position = 0
        #legend_y_position = 0
        #subtract_from_y = 2
        
        #for ITEM in range(0, len(items_for_legend[number_of_tab]), 7):
            
        #    legend = Legend(items=items_for_legend[number_of_tab][ITEM:ITEM+7],
        #             location=(legend_x_position, legend_y_position), orientation="vertical")
        #    p.add_layout(legend, 'below')
        #    legend_x_position = legend_x_position + 100
        #    legend_y_position = legend_y_position + 200 - subtract_from_y
            
        #p.legend.label_text_font_size = '10pt'
        #p.legend.click_policy="hide"
        
        # Create legends
        """
        legend_x_position = 0
        legend_y_position = 0
        subtract_from_y = 2
        
        legend_linear=[]

        for ITEM in range(0, len(items_for_legend[number_of_tab]), 7):
            legend_linear.append(Legend(items=items_for_legend[number_of_tab][ITEM:ITEM+7],
                      location=(legend_x_position, legend_y_position), orientation="vertical"))
            p.add_layout(legend_linear[-1], 'below')
            legend_x_position = legend_x_position + 100
            legend_y_position = legend_y_position + 200 - subtract_from_y

        for lg in legend_linear:
            lg.label_text_font_size = '10pt'
            lg.click_policy = "hide"
            lg.visible = True
        """
        #######
        
        # renderers, table for all coordinates
    
        dict_of_sources_non_linear[number_of_tab] = {}
        dict_for_renderers_non_linear[number_of_tab] = []
        
        ordered = pd.DataFrame()
        clusters_ordered = []
        ascii_ = []
        if type(list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab][0]) == str:
            for i in list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab]:
                sum_ascii = 0
                for j in i:
                    sum_ascii = sum_ascii + ord(j)
                ascii_.append(sum_ascii)
                clusters_ordered.append(i)
            ordered['Clusters']=clusters_ordered
            ordered['ASCII values']=ascii_
            ordered = ordered.sort_values('ASCII values')
            ordered.head(40)
            list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab] = ordered['Clusters'].to_list()
                
        if type(list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab][0]) == int or type(list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab][0]) == float:
            list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab].sort()

        combined_data.append({
            'x': [],
            'y': [],
            'original_x': [],
            'original_y': [],
            'click_count': [],
            'locked': [],
            'color': [],
            'clusters':[],
            'in_cdsview':[],
            'CellIDs': [],
            'size': []
        })
        
        for index, i in enumerate(list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab]):
            if type(i) == int or type(i) == float:
                dict_of_sources_non_linear[number_of_tab]["C{}".format(i)] = ColumnDataSource(data=dict(x=list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['x'], 
                                                                      y=list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['y'],
                                                                      locked=[False] * len(list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['y']),
                                                              original_x=list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['x'].copy(), 
                                                               original_y=list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['y'].copy(),
                                                                click_count=[0] * len(list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['y']),
                                                                in_cdsview=list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['in_cdsview'],
                                                                clusters=list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['clusters'],
                                                                 CellIDs=list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['CellIDs'],
                                                                size=[1] * len(list_for_all_data_types_remaining[number_of_tab]["C_{}".format(i)]['y'])))

                boolean_filter = BooleanFilter(dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['in_cdsview'])
                view = CDSView(filter=boolean_filter)

                #R = p.scatter(x='x', y='y', source=dict_of_sources_non_linear[number_of_tab]["C{}".format(i)], view=view, color=colors_non_linear[index], size=1, visible=False)

                #dict_for_renderers_non_linear[number_of_tab].append(R)

                #items_for_legend_non_linear[number_of_tab].append(LegendItem(label="C_{}".format(i), renderers=[R]))

                combined_data[number_of_tab]['x'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['x']
                combined_data[number_of_tab]['y'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['y']
                combined_data[number_of_tab]['original_x'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['original_x']
                combined_data[number_of_tab]['original_y'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['original_y']
                combined_data[number_of_tab]['click_count'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['click_count']
                combined_data[number_of_tab]['locked'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['locked']
                combined_data[number_of_tab]['clusters'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['clusters']
                combined_data[number_of_tab]['in_cdsview'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['in_cdsview']
                combined_data[number_of_tab]['CellIDs'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['CellIDs']
                combined_data[number_of_tab]['size'] += dict_of_sources_non_linear[number_of_tab]["C{}".format(i)].data['size']
                
                

            if type(i) == str:
                
                dict_of_sources_non_linear[number_of_tab][i] = ColumnDataSource(data=dict(x=list_for_all_data_types_remaining[number_of_tab][i]['x'], 
                                                        y=list_for_all_data_types_remaining[number_of_tab][i]['y'],
                                                        locked=[False] * len(list_for_all_data_types_remaining[number_of_tab][i]['y']),
                                                        original_x=list_for_all_data_types_remaining[number_of_tab][i]['x'].copy(), 
                                                        original_y=list_for_all_data_types_remaining[number_of_tab][i]['y'].copy(),
                                                        click_count=[0] * len(list_for_all_data_types_remaining[number_of_tab][i]['y']),
                                                        in_cdsview=list_for_all_data_types_remaining[number_of_tab][i]['in_cdsview'],
                                                        clusters=list_for_all_data_types_remaining[number_of_tab][i]['clusters'],
                                                        CellIDs=list_for_all_data_types_remaining[number_of_tab][i]['CellIDs'],
                                                        size=[1] * len(list_for_all_data_types_remaining[number_of_tab][i]['y'])))

                
                boolean_filter = BooleanFilter(dict_of_sources_non_linear[number_of_tab][i].data['in_cdsview'])
                view = CDSView(filter=boolean_filter)
                
                #R = p.scatter(x='x', y='y', source=dict_of_sources_non_linear[number_of_tab][i], view=view, color=colors_non_linear[index], size=1, visible=False)

                #dict_for_renderers_non_linear[number_of_tab].append(R)

                #items_for_legend_non_linear[number_of_tab].append(LegendItem(label=i, renderers=[R]))

                combined_data[number_of_tab]['x'] += dict_of_sources_non_linear[number_of_tab][i].data['x']
                combined_data[number_of_tab]['y'] += dict_of_sources_non_linear[number_of_tab][i].data['y']
                combined_data[number_of_tab]['original_x'] += dict_of_sources_non_linear[number_of_tab][i].data['original_x']
                combined_data[number_of_tab]['original_y'] += dict_of_sources_non_linear[number_of_tab][i].data['original_y']
                combined_data[number_of_tab]['click_count'] += dict_of_sources_non_linear[number_of_tab][i].data['click_count']
                combined_data[number_of_tab]['locked'] += dict_of_sources_non_linear[number_of_tab][i].data['locked']
                combined_data[number_of_tab]['clusters'] += dict_of_sources_non_linear[number_of_tab][i].data['clusters']
                combined_data[number_of_tab]['in_cdsview'] += dict_of_sources_non_linear[number_of_tab][i].data['in_cdsview']
                combined_data[number_of_tab]['CellIDs'] += dict_of_sources_non_linear[number_of_tab][i].data['CellIDs']
                combined_data[number_of_tab]['size'] += dict_of_sources_non_linear[number_of_tab][i].data['size']
                    
        # Create a new ColumnDataSource with the combined data
        combined_source.append(ColumnDataSource(data=dict(
            x= combined_data[number_of_tab]['x'],
            y= combined_data[number_of_tab]['y'],
            original_x= combined_data[number_of_tab]['original_x'],
            original_y= combined_data[number_of_tab]['original_y'],
            click_count= combined_data[number_of_tab]['click_count'],
            locked= combined_data[number_of_tab]['locked'],
            clusters=combined_data[number_of_tab]['clusters'],
            in_cdsview=combined_data[number_of_tab]['in_cdsview'],
            CellIDs=combined_data[number_of_tab]['CellIDs'],
            size=combined_data[number_of_tab]['size']
        )))

        boolean_filter = BooleanFilter(combined_source[number_of_tab].data['in_cdsview'])
        view = CDSView(filter=boolean_filter)

        for i in dict_for_renderers[number_of_tab]:
            spinners_for_all.append(spinners_for_all_data_point_size[number_of_tab].js_link('value', i.glyph, 'size'))
       
        R = p.scatter(x='x', y='y', source=combined_source[number_of_tab], size='size', visible=False, view=view,
                      color=factor_cmap('clusters', colors[number_of_tab], list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab]))

        R.nonselection_glyph.hatch_alpha = 1
        R.nonselection_glyph.line_alpha = 1
        R.nonselection_glyph.fill_alpha = 1
        R.nonselection_glyph.size = 1

        R.glyph.hatch_alpha = 1
        R.glyph.line_alpha = 1
        R.glyph.fill_alpha = 1
        
        for i in dict_for_renderers_non_linear[number_of_tab]:
            spinners_for_all_non_linear.append(spinners_for_all_data_point_size_non_linear[number_of_tab].js_link('value', R.glyph, 'size'))
 
        dict_for_renderers_non_linear[number_of_tab].append(R)
        
        data_point_size_callback_for_non_selection_glyphs = CustomJS(args=dict(renderer=dict_for_renderers_non_linear[number_of_tab][0],
                                                                              point_size=spinners_for_all_data_point_size_non_linear[number_of_tab]), code="""

            renderer.nonselection_glyph.size = point_size.value;
            renderer.glyph.size = point_size.value;

            renderer.nonselection_glyph.hatch_alpha = point_size.value;
            renderer.nonselection_glyph.line_alpha = point_size.value;
            renderer.nonselection_glyph.fill_alpha = point_size.value;
            renderer.nonselection_glyph.size = point_size.value;
    
            renderer.glyph.hatch_alpha = point_size.value;
            renderer.glyph.line_alpha = point_size.value;
            renderer.glyph.fill_alpha = point_size.value;

            renderer.change.emit();
        """)

        spinners_for_all_data_point_size_non_linear[number_of_tab].js_on_change('value', data_point_size_callback_for_non_selection_glyphs)
        
        #color=factor_cmap('clusters', 'Category20_3', list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab]))
        #color=factor_cmap('clusters', colors, list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab])
        
        #dict_for_renderers_non_linear[number_of_tab].append(R)

        #legend_items = []
        #for cluster, color in zip(list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab], colors_non_linear):
        #    items_for_legend_non_linear[number_of_tab].append(LegendItem(label=cluster, renderers=[R], index=list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab].index(cluster)))

        #for i in list_for_storing_total_clusters_info_in_each_file_remaining[number_of_tab]:
        #    items_for_legend_non_linear[number_of_tab].append(LegendItem(label=i, renderers=[R]))
        
        #legend_x_position = 0
        #legend_y_position = 0
        #subtract_from_y = 2

        #for ITEM in range(0, len(items_for_legend_non_linear[number_of_tab]), 7):
        #    legend_non_linear.append(Legend(items=items_for_legend_non_linear[number_of_tab][ITEM:ITEM+7],
        #              location=(legend_x_position, legend_y_position), orientation="vertical"))
        #    p.add_layout(legend_non_linear[-1], 'below')
        #    legend_x_position = legend_x_position + 100
        #    legend_y_position = legend_y_position + 200 - subtract_from_y#

        #for lg in legend_non_linear:
            #lg.label_text_font_size = '10pt'
            #lg.click_policy = "hide"
            #lg.visible = False

        draw_tool.append(PointDrawTool(renderers=dict_for_renderers_non_linear[number_of_tab], add=False))
        p.add_tools(draw_tool[number_of_tab])
        
        #########
        
        dict_of_coordinates_for_source_final["xcoord"][number_of_tab] = []
        dict_of_coordinates_for_source_final["ycoord"][number_of_tab] = []
        for key, value in dict_of_sources[number_of_tab].items():
            dict_of_coordinates_for_source_final["xcoord"][number_of_tab].append(value.data['x'])
            dict_of_coordinates_for_source_final["ycoord"][number_of_tab].append(value.data['y'])
        dict_of_coordinates_for_source_final["xcoord"][number_of_tab] = [item for sublist in dict_of_coordinates_for_source_final["xcoord"][number_of_tab] for item in sublist]
        dict_of_coordinates_for_source_final["ycoord"][number_of_tab] = [item for sublist in dict_of_coordinates_for_source_final["ycoord"][number_of_tab] for item in sublist]    
        
        list_of_source_final.append(ColumnDataSource(data=dict(x=dict_of_coordinates_for_source_final["xcoord"][number_of_tab], 
                                                               y=dict_of_coordinates_for_source_final["ycoord"][number_of_tab])))
        
        columns = [TableColumn(field="x", title="x"), TableColumn(field="y", title="y")]

        list_for_data_table.append(DataTable(source=list_of_source_final[number_of_tab], columns=columns, 
                                             editable=True, height=200, visible=visibility))

        dict_for_active_array[number_of_tab] = []
        for i in range(len(list_for_storing_total_clusters_info_in_each_file[number_of_tab])):
            dict_for_active_array[number_of_tab].append(i)    

            
        div_cluster_heading.append(Div(text="""Select/Unselect combinations of clusters based on whether a distinct pattern is visible (Unselected clusters can be selected again by clicking on the corresponding legend): """, 
                             visible=visibility, width=600, sizing_mode="stretch_height", height=15))

        div_transform_heading.append(Div(text="""Type of Transformations: """, 
                             visible=visibility, width=600, sizing_mode="stretch_height", height=15))
            
        dict_for_threshold["spinners"].append(Spinner(title="Threshold:", low=1, high=5000, 
                                                      step=1, value=1000, width=200, visible=False))

        list_of_linear_buttons.append(Button(label="Linear Transformations", button_type="primary",
                                                                 width = 250, visible=visibility))
        
        list_of_non_linear_buttons.append(Button(label="Non-linear Transformations", button_type="default",
                                                                 width = 250, visible=visibility))
        
        tap_callback = CustomJS(args=dict(source=combined_source[number_of_tab],
                                        point_size=spinners_for_all_data_point_size_non_linear[number_of_tab]), code="""

            const selected_indices = source.selected.indices;
            const sizes = source.data['size'];
            const locked = source.data['locked'];
        
            if (selected_indices.length > 0) {
                const index = selected_indices[0];
                const click_count = source.data['click_count'][index];
        
                // Toggle locked state on every second click
                if (click_count % 2 === 0) {
                    locked[index] = !locked[index];
                }

                source.data['click_count'][index] += 1;
               
                // Emit change for the updated data source
                source.change.emit();
            }

        """)
        
        drag_callback = CustomJS(args=dict(threshold_for_distortion=dict_for_threshold["spinners"][number_of_tab], 
                                   sources=combined_source[number_of_tab]), code="""

                var threshold = threshold_for_distortion.value;
                               
                const selected_indices = sources.selected.indices;
            
                if (selected_indices.length > 0) {
                    const original_x = sources.data['original_x'];
                    const original_y = sources.data['original_y'];
                    const x = sources.data['x'];
                    const y = sources.data['y'];
                    const click_count = sources.data['click_count'];
                    const locked_renderers = sources.data['locked'];
                    const index = selected_indices[0];
                    
                    click_count[index] += 1;
            
                    // Calculate dx and dy
                    const dx = x[index] - original_x[index];
                    const dy = y[index] - original_y[index];
            
                    // Update positions of neighbors
                    const thresholdSquared = threshold * threshold;
            
                    for (let i = 0; i < x.length; i++) {
            
                        const distanceSquared = Math.pow(original_x[i] - original_x[index], 2) +
                                                Math.pow(original_y[i] - original_y[index], 2);
            
                        if (distanceSquared < thresholdSquared) {
                            const factor = (threshold - Math.sqrt(distanceSquared)) / threshold;
                            x[i] = original_x[i] + dx * factor;
                            y[i] = original_y[i] + dy * factor;
                        }
                    }
            
                    // Emit change for the updated data source
                    sources.change.emit();
                }
            """)

        #dict_for_threshold["spinners"][number_of_tab].js_on_change('value', tap_callback)
        dict_for_threshold["spinners"][number_of_tab].js_on_change('value', drag_callback)
        
        #p.toolbar.active_tap = draw_tool
        #p.js_on_event(Tap, tap_callback)
        p.js_on_event(MouseMove, tap_callback)
        p.js_on_event(MouseMove, drag_callback)
        
        # multichoice - for finding patterns
        
        #select_dict['dropdown'].append(Select(title="Type of Transformations:", value="Linear Transformations", 
        #        options=["Linear Transformations", "Non-linear Transformations"]))

        #dict_for_multichoice["multichoice"].append(MultiChoice(value=list_for_storing_total_clusters_info_in_each_file[number_of_tab], 
        #                                                options=list_for_storing_total_clusters_info_in_each_file[number_of_tab],
        #                                                visible=visibility))
        
        # image gallery
        dict_for_img_number["sliders"].append(Slider(start=0, end=10, value=0, step=1, title="Image-Number",
                                                     visible=False))
        dict_for_img_number["spinners"].append(Spinner(title="Image Number:", low=0, high=len(imgs)-1, 
                                                      step=1, value=0, width=200, visible=visibility))

        # pixel size for h&e image
        dict_for_he_pixel_size["sliders"].append(Slider(start=0.01, end=3, value=scale_for_he_image, step=0.01, 
                                                        title="Pixel Size (in microns/pixel) for H&E image (lowest: 0.01, highest: 3):",
                                                        visible=False))
        dict_for_he_pixel_size["spinners"].append(Spinner(title="H&E Image:", 
                                                         low=0.01, high=3, step=0.01, value=scale_for_he_image, 
                                                          width=200, visible=visibility))

        

        # rotation for data
        dict_for_data_rot_angle["sliders"].append(Slider(start=-360, end=360, value=0, 
                                                         step=1, title="Rotation Angle for Data (in degrees) (lowest: -360, highest: 360):",
                                                        visible=False))
        dict_for_data_rot_angle["spinners"].append(Spinner(title="Data:", 
                           low=-360, high=360, step=1, value=0, width=200, visible=visibility))

        # pixel size for data
        dict_for_data_pixel_size["sliders"].append(Slider(start=0.01, end=3, value=scale_for_sm_data[number_of_tab], step=0.01, 
                                                          title="Pixel Size for Data (in um/px) (lowest: 0.01, highest: 3):",visible=False))
        dict_for_data_pixel_size["spinners"].append(Spinner(title="Data:",
                                                            low=0.01, high=3, step=0.01, value=scale_for_sm_data[number_of_tab],
                                                            width=200
                                                           ,visible=visibility))

        # pixel size for scale bar
        dict_for_scale_bar_pixel_size["sliders"].append(Slider(start=0.01, end=3, value=scale_bar[number_of_tab], step=0.01, 
                                                          title="Pixel Size for Scale Bar (in um/px) (lowest: 0.01, highest: 3):",visible=False))
        dict_for_scale_bar_pixel_size["spinners"].append(Spinner(title="Display Scale:",
                                                            low=0.01, high=3, step=0.01, value=scale_bar[number_of_tab], width=200
                                                           ,visible=visibility))        

        # for changing orientation of data
        dict_for_toggle_vert["toggle_buttons"].append(Toggle(label="Flip about Vertical Plane", button_type="success", width = 250,
                                                            visible=visibility))

        dict_for_toggle_hori["toggle_buttons"].append(Toggle(label="Flip about Horizontal Plane", button_type="success", width = 250,
                                                            visible=visibility))
        
        # for saving modified coordinates
        dict_for_data_save_button["save_buttons"].append(Button(label="Download Spatial Coordinates+Log file", 
                                                                button_type="success",width=250,visible=visibility))  

        # rotation for h&e image
        dict_for_he_rot_angle["sliders"].append(Slider(start=-360, end=360, 
                                                       value=0, step=1, title="Image-Rotation in degrees (lowest: -360, highest: 360)",visible=False))
        dict_for_he_rot_angle["spinners"].append(Spinner(title="H&E Image:", 
                                                         low=-360, high=360, step=1, value=0, width=200,visible=visibility))
        
        
        dict_for_image_save_button["save_buttons"].append(Button(label="Download Updated Image", button_type="success",
                                                                 width = 250, visible=visibility))
        
        
        #dict_for_data_translocation_X["sliders"].append(Slider(value=0, step=1, 
        #                                                  title="Data Transloaction - X coordinate:",visible=False))
        
        dict_for_data_translocation_X["spinners"].append(Spinner(title="x coordinate:", step=1, 
                                                                 value=0, width=200, visible=visibility))
        
        #dict_for_data_translocation_Y["sliders"].append(Slider(value=0, step=1,
        #                                                  title="Data Transloaction - Y coordinate:",visible=False))
      
        dict_for_data_translocation_Y["spinners"].append(Spinner(title="y coordinate:", step=1, 
                                                                 value=0, width=200,visible=visibility))
        
        
        modifications = ['vertical flip', 'horizontal flip', 'data pixel size', 'image pixel size', 'scale bar pixel size', 'data rotation', 'image rotation', 'translocation in x-axis', 'translocation in y-axis', 'image number']
        
        modifications_values = ['false', 'false', scale_for_sm_data[number_of_tab], scale_for_he_image, scale_bar[number_of_tab], 0, 0, 0, 0, 0]
        
        undo_dict[number_of_tab] = ColumnDataSource(data=dict(method=[], value=[]))
        
        redo_dict[number_of_tab] = ColumnDataSource(data=dict(method=[], value=[]))
        identifier[number_of_tab] = ColumnDataSource(data=dict(count=[0], track_flips=[0], undo_in_progress=[False], redo_in_progress=[False]))

        for name, value in zip(modifications, modifications_values):
            log_file[number_of_tab].data['method'].append(name)
            log_file[number_of_tab].data['value'].append(value)
            
            undo_dict[number_of_tab].data['method'].append(name)
            undo_dict[number_of_tab].data['value'].append(value)
            
        
        dict_undo_button['buttons'].append(Button(label="Undo", button_type="success",
                                                                 width = 250, visible=visibility))
        
        dict_redo_button['buttons'].append(Button(label="Redo", button_type="success",
                                                                 width = 250, visible=visibility))
 
        dict_for_widget_names_function_names[number_of_tab] = {}
        dict_for_widget_names_function_names[number_of_tab]['image number']=dict_for_img_number["sliders"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['image pixel size']=dict_for_he_pixel_size["sliders"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['scale bar pixel size']=dict_for_scale_bar_pixel_size["sliders"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['data rotation']=dict_for_data_rot_angle["sliders"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['data pixel size']=dict_for_data_pixel_size["sliders"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['vertical flip']=dict_for_toggle_vert["toggle_buttons"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['horizontal flip']=dict_for_toggle_hori["toggle_buttons"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['image rotation']=dict_for_he_rot_angle["sliders"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['translocation in x-axis']=dict_for_data_translocation_X["spinners"][number_of_tab]
        dict_for_widget_names_function_names[number_of_tab]['translocation in y-axis']=dict_for_data_translocation_Y["spinners"][number_of_tab]
        
        dict_undo_button['buttons'][number_of_tab].js_on_click(CustomJS(args=dict(widget_stock=dict_for_widget_names_function_names[number_of_tab],
                                                 undo_file=undo_dict[number_of_tab], 
                                                 redo_file=redo_dict[number_of_tab],
                                                 iden = identifier[number_of_tab],
                                                 log__file = log_file[number_of_tab]),
                            code=""" 
                            
                        function logState(label) {
                            console.log(`${label} - undo_file methods:`, undo_file.data['method']);
                            console.log(`${label} - undo_file values:`, undo_file.data['value']);
                            
                            console.log(`${label} - redo_file methods:`, redo_file.data['method']);
                            console.log(`${label} - redo_file values:`, redo_file.data['value']);
                        }

                        function performPop(arrayObj) {
                            arrayObj.data['method'].pop();
                            arrayObj.data['value'].pop();
                            arrayObj.change.emit();
                        }

                        function performPush(arrayObj, method, value) {
                            arrayObj.data['method'].push(method);
                            arrayObj.data['value'].push(value);
                            arrayObj.change.emit();
                        }
                        
                        function undoOperation() {
                            
                            if (undo_file.data['method'].length === 10) {
                                iden.data['count'][0] = 0;
                                iden.change.emit();
                            }

                            if (undo_file.data['method'].length > 10) {
                                
                                iden.data['undo_in_progress'][0] = true; // Set the undo in progress flag
                                iden.change.emit();
                                
                                console.log("inside undo");
                                console.log(iden.data['undo_in_progress'][0]);
                                
                                let last_method_of_undo = undo_file.data['method'][undo_file.data['method'].length - 1];
                                let secondlast_method_of_undo = undo_file.data['method'][undo_file.data['method'].length - 2];

                                if (last_method_of_undo === 'data rotation' && secondlast_method_of_undo.includes(' flip')) {
                                    console.log("inside rotation_flip_if_statement");
                                    
                                    last_method_of_undo = undo_file.data['method'][undo_file.data['method'].length - 1];
                                    let last_value_of_undo = undo_file.data['value'][undo_file.data['value'].length - 1];
                                    performPush(redo_file, last_method_of_undo, last_value_of_undo);
                                    
                                    performPop(undo_file);

                                    last_method_of_undo = undo_file.data['method'][undo_file.data['method'].length - 1];
                                    last_value_of_undo = undo_file.data['value'][undo_file.data['value'].length - 1];
                                    performPush(redo_file, last_method_of_undo, last_value_of_undo);

                                    iden.data['track_flips'][0] = 1;
                                    iden.change.emit();
                                } else {
                                    if (iden.data['track_flips'][0] === 1) {
                                        console.log("inside rotation_flip_else_if statement");
                                        performPop(undo_file);

                                        iden.data['track_flips'][0] = 0;
                                        iden.change.emit();

                                        last_method_of_undo = undo_file.data['method'][undo_file.data['method'].length - 1];
                                        let last_value_of_undo = undo_file.data['value'][undo_file.data['value'].length - 1];
                                        performPush(redo_file, last_method_of_undo, last_value_of_undo);
                                    } 
                                    
                                    else {
                                        console.log("inside rotation_flip_else_else statement");
                                        last_method_of_undo = undo_file.data['method'][undo_file.data['method'].length - 1];
                                        let last_value_of_undo = undo_file.data['value'][undo_file.data['value'].length - 1];
                                        performPush(redo_file, last_method_of_undo, last_value_of_undo);
                                    }
                                }

                                let method_lastIndex = undo_file.data['method'].lastIndexOf(last_method_of_undo);
                                let method_secondLastIndex = undo_file.data['method'].lastIndexOf(last_method_of_undo, method_lastIndex - 1);

                                if (last_method_of_undo === "vertical flip" || last_method_of_undo === "horizontal flip") {
                                    console.log("inside flip statement");
                                    let for_toggles = undo_file.data['value'][method_secondLastIndex].toLowerCase() === 'true';
                                    
                                    widget_stock[last_method_of_undo].active = for_toggles;
                                    
                                } else {
                                    let for_others = undo_file.data['value'][method_secondLastIndex];
                                    
                                    widget_stock[last_method_of_undo].value = for_others;
                                    
                                }

                                performPop(undo_file);
                                
                                iden.data['count'][0] = 1;
                                //iden.data['undo_in_progress'][0] = false; // Reset the undo in progress flag
                                iden.change.emit();
                                console.log("outside undo loops, still in undo function");
                                console.log(iden.data['undo_in_progress'][0]);
                                }
                            }

                            // Example usage
                            console.log("start of callback");
                            logState("Initial");

                            // Perform undo operation
                            undoOperation();

                            logState("After undoOperation");
                            console.log("end of callback");
                            
                            
                            """))
        
        callback_for_undo_button_REDO = CustomJS(args=dict(widget_stock=dict_for_widget_names_function_names[number_of_tab],
                                                 undo_file=undo_dict[number_of_tab], 
                                                 redo_file=redo_dict[number_of_tab],
                                                 iden = identifier[number_of_tab]),
                            code=""" 
                            
                            function logState(label) {
                                console.log(`${label} - redo_file methods:`, redo_file.data['method']);
                                console.log(`${label} - redo_file values:`, redo_file.data['value']);
                            }

                            function performPop(arrayObj) {
                                arrayObj.data['method'].pop();
                                arrayObj.data['value'].pop();
                                arrayObj.change.emit();
                            }

                            function performPush(arrayObj, method, value) {
                                arrayObj.data['method'].push(method);
                                arrayObj.data['value'].push(value);
                                arrayObj.change.emit();
                            }
                            
                            function redoOperation() {
                                logState("Before redoOperation");
                                
                                console.log(`undo_file methods:`, undo_file.data['method']);
                                console.log(`undo_file values:`, undo_file.data['value']);

                                if (redo_file.data['method'].length > 0) {
                                    
                                    iden.data['redo_in_progress'][0] = true; 
                                    iden.change.emit();
                                
                                    let last_method_of_redo = redo_file.data['method'][redo_file.data['method'].length - 1];
                                    let last_value_of_redo = redo_file.data['value'][redo_file.data['value'].length - 1];
                                    let secondlast_method_of_redo = redo_file.data['method'][redo_file.data['method'].length - 2];

                                    performPush(undo_file, last_method_of_redo, last_value_of_redo);

                                    if (widget_stock.hasOwnProperty(last_method_of_redo)) {
                                    
                                    if (last_method_of_redo.includes(' flip') && secondlast_method_of_redo === 'data rotation') {
                                    
                                    console.log("inside flip redo statement");
                                    widget_stock[last_method_of_redo].active = last_value_of_redo;
                                    performPop(redo_file);
                                    
                                    last_method_of_redo = redo_file.data['method'][redo_file.data['method'].length - 1];
                                    last_value_of_redo = redo_file.data['value'][redo_file.data['value'].length - 1];

                                    performPush(undo_file, last_method_of_redo, last_value_of_redo);
                                    performPop(redo_file);
                                    
                                    iden.data['track_flips'][0] = 1;
                                    iden.change.emit();
                                    }
                                    
                                    else {
                                        //let method_lastIndex = undo_file.data['method'].lastIndexOf(last_method_of_redo);
                                        //let for_others = redo_file.data['value'][method_lastIndex];
                                        widget_stock[last_method_of_redo].value = last_value_of_redo;
                                        performPop(redo_file);
                                    }
                                    } else {
                                        console.error(`Invalid method: ${last_method_of_redo}`);
                                    }
                                }
                                
                                logState("After redoOperation");
                                console.log(`undo_file methods:`, undo_file.data['method']);
                                console.log(`undo_file values:`, undo_file.data['value']);
                            }

                            // Perform redo operation
                            redoOperation();
                            
                            """)
        
        dict_redo_button['buttons'][number_of_tab].js_on_click(callback_for_undo_button_REDO)
        
        #dict_for_img_number["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        #dict_for_he_pixel_size["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        #dict_for_scale_bar_pixel_size["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        #dict_for_data_rot_angle["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        #dict_for_data_pixel_size["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        #dict_for_toggle_vert["toggle_buttons"][number_of_tab].js_on_event(ButtonClick, callback_for_other_widgets_REDO)
        #dict_for_toggle_hori["toggle_buttons"][number_of_tab].js_on_event(ButtonClick, callback_for_other_widgets_REDO)
        #dict_for_he_rot_angle["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        #dict_for_data_translocation_X["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        #dict_for_data_translocation_Y["spinners"][number_of_tab].js_on_event(ValueSubmit, callback_for_other_widgets_REDO)
        
        # callbacks
        #dict_for_multichoice["callback"] = CustomJS(args=dict(renderers=dict_for_renderers[number_of_tab],
        #                                                     multichoice=dict_for_multichoice["multichoice"][number_of_tab], 
        #                                                     table=list_for_data_table[number_of_tab],
        #                                                     all_clusters = list_for_storing_total_clusters_info_in_each_file[number_of_tab]),
        #                    code=""" 

        #    var xval = table.source.data['x'];
        #    var yval = table.source.data['y'];
        #    var selected_vals = multichoice.value;

        #    for (var i = 0; i < renderers.length; i++){
        #        renderers[i].glyph.fill_alpha=0;
        #        renderers[i].glyph.line_alpha=0;}

        #   var j=0;
        #    for (var i = 0; i < selected_vals.length; i++){
        #    for (var h = 0; h < all_clusters.length; h++){
        #     if (all_clusters[h] == selected_vals[i]){

        #     renderers[h].glyph.fill_alpha=1;
        #     renderers[h].glyph.line_alpha=1;

        #      for (var z = 0; z < renderers[h].data_source.data['x'].length; z++){
        #          xval[j] = renderers[h].data_source.data['x'][z];
        #          yval[j] = renderers[h].data_source.data['y'][z];
        #          j++;

        #     }}}}

        #     //if (j < xval.length){
        #     //for (j; j < xval.length; j++){
        #     //xval[j] = 'NaN';
        #     //yval[j] = 'NaN';}
        #     //}

        #     table.change.emit();
        #""")
        #dict_for_multichoice["multichoice"][number_of_tab].js_on_change('value', dict_for_multichoice["callback"])
        
        dict_for_img_number["callback"] = CustomJS(args=dict(width_of_images=width_of_images,
                                       height_of_images=height_of_images, 
                                       px_size=dict_for_he_pixel_size["sliders"][number_of_tab], 
                                       source=source_img,
                                       im=im, 
                                       iden = identifier[number_of_tab],
                                       log__file = log_file[number_of_tab],   
                                       undo_file = undo_dict[number_of_tab],
                                       redo_file = redo_dict[number_of_tab],
                                       image_number = dict_for_img_number["sliders"][number_of_tab], 
                                       division_factor_for_resize_he=division_factor_for_resize_he,
                                       division_factor_for_resize_scale_bar=division_factor_for_resize_scale_bar,
                                       scale_bar_px_size = dict_for_scale_bar_pixel_size["sliders"][number_of_tab]),
                            code="""
                            
                            const P = px_size.value;
                            const N = image_number.value;
                            const S = scale_bar_px_size.value;
                            
                            var num_fac_scale_bar = division_factor_for_resize_scale_bar.data['x'];
                            var den_fac_scale_bar = division_factor_for_resize_scale_bar.data['y'];
                            var resize_fac_scale_bar = num_fac_scale_bar[0]/den_fac_scale_bar[0];
                            //num_fac_scale_bar[0] = 1;
                            //den_fac_scale_bar[0] = S;

                            im.data_source.data['url'] = [source.data['imgs'][N]];
                            im.data_source.change.emit();

                            var num_fac = division_factor_for_resize_he.data['x'];
                            var den_fac = division_factor_for_resize_he.data['y'];
                            var resize_fac = num_fac[N]/den_fac[N];
                            num_fac[N] = P;
                            den_fac[N] = 1;

                            width_of_images[N] = (width_of_images[N] / P) * resize_fac * resize_fac_scale_bar * S;
                            height_of_images[N] = (height_of_images[N] / P) * resize_fac * resize_fac_scale_bar * S;

                            im.glyph.x = width_of_images[N]/2;
                            im.glyph.y = height_of_images[N]/2;
                            im.glyph.w = width_of_images[N];
                            im.glyph.h = height_of_images[N];
                            im.glyph.anchor = 'center';

                            im.glyph.change.emit();
                            division_factor_for_resize_he.change.emit();
                            division_factor_for_resize_scale_bar.change.emit();
                            
                            function handleNewWidgetOperation() {
                
                                redo_file.data['method'] = [];
                                redo_file.data['value'] = [];
                                redo_file.change.emit();
                            }

                            if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
                                
                                var last_index_img_pixel_size = log__file.data['method'].lastIndexOf('image pixel size');
                            
                                if (log__file.data['value'][last_index_img_pixel_size] != P){
                                
                                log__file.data['method'].push('image pixel size');
                                log__file.data['value'].push(P);
                                log__file.change.emit();

                                undo_file.data['method'].push('image pixel size');
                                undo_file.data['value'].push(P);
                                undo_file.change.emit();

                                }
                                
                                var last_index_img_number = log__file.data['method'].lastIndexOf('image number');
                                if (log__file.data['value'][last_index_img_number] != N){
                                
                                log__file.data['method'].push('image number');
                                log__file.data['value'].push(N);
                                log__file.change.emit();

                                undo_file.data['method'].push('image number');
                                undo_file.data['value'].push(N);
                                undo_file.change.emit();

                                }
                                
                                if (log__file.data['value'][last_index_img_number] != N && log__file.data['value'][last_index_img_pixel_size] != P) {
                                
                                handleNewWidgetOperation();
                                
                                }
                            }
                            
                            else {
                                
                                var last_index_img_pixel_size = log__file.data['method'].lastIndexOf('image pixel size');
                            
                                if (log__file.data['value'][last_index_img_pixel_size] != P){
                                log__file.data['method'].push('image pixel size');
                                log__file.data['value'].push(P);
                                log__file.change.emit();

                                }
                                
                                var last_index_img_number = log__file.data['method'].lastIndexOf('image number');
                                if (log__file.data['value'][last_index_img_number] != N){

                                log__file.data['method'].push('image number');
                                log__file.data['value'].push(N);
                                log__file.change.emit();

                                }
                            
                                iden.data['undo_in_progress'][0] = false; 
                                iden.change.emit();
                            }
                            
                            // for redo
            
                            if (iden.data['redo_in_progress'][0]) {
                                var last_index_img_pixel_size = log__file.data['method'].lastIndexOf('image pixel size');
                            
                                if (log__file.data['value'][last_index_img_pixel_size] != P){
                                log__file.data['method'].push('image pixel size');
                                log__file.data['value'].push(P);
                                log__file.change.emit();

                                }
                                
                                var last_index_img_number = log__file.data['method'].lastIndexOf('image number');
                                if (log__file.data['value'][last_index_img_number] != N){

                                log__file.data['method'].push('image number');
                                log__file.data['value'].push(N);
                                log__file.change.emit();

                                }
                            iden.data['redo_in_progress'][0] = false; 
                            iden.change.emit();
                            }
                            
        """)

        dict_for_img_number["spinners"][number_of_tab].js_link('value', dict_for_img_number["sliders"][number_of_tab], 'value')
        dict_for_img_number["sliders"][number_of_tab].js_link('value', dict_for_img_number["spinners"][number_of_tab], 'value')
        dict_for_img_number["sliders"][number_of_tab].js_on_change('value', dict_for_img_number["callback"])

        dict_for_he_pixel_size["spinners"][number_of_tab].js_link('value', dict_for_he_pixel_size["sliders"][number_of_tab], 'value')
        dict_for_he_pixel_size["sliders"][number_of_tab].js_link('value', dict_for_he_pixel_size["spinners"][number_of_tab], 'value')
        dict_for_he_pixel_size["sliders"][number_of_tab].js_on_change('value', dict_for_img_number["callback"])

        dict_for_data_rot_angle["callback"] = CustomJS(args=dict(table=list_for_data_table[number_of_tab], 
                                            renderers=dict_for_renderers[number_of_tab], 
                                      rot=dict_for_data_rot_angle["sliders"][number_of_tab],
                                     rotation_values=list_for_rotation_values[number_of_tab],
                                     log__file = log_file[number_of_tab],
                                      iden = identifier[number_of_tab],
                                      undo_file = undo_dict[number_of_tab],
                                      redo_file = redo_dict[number_of_tab]),
                            code="""

            const V = rot.value;
            var rotval = rotation_values.data['rotval'];
            const A = V - rotval[0];
            
            const radians = (Math.PI / 180) * A;
            const cos = Math.cos(radians);
            const sin = Math.sin(radians);

            var x = table.source.data['x'];
            var y = table.source.data['y'];
            var x_clean = [];
            var y_clean = [];

            for (var i = 0; i < x.length; i++){
            if (x[i] != 'NaN'){
            x_clean[i]=x[i];
            y_clean[i]=y[i]; }}

            var w = rotation_values.data['x'];
            var h = rotation_values.data['y'];

            var X=0;
            var Y=0;
            var x_rot = [];
            var y_rot = [];

            console.log("inside data rotation, before rotation");
            console.log(x[0]);
            
            for (var i = 0; i < x_clean.length; i++) {
                X= x_clean[i];
                Y= y_clean[i];
                x[i] = ((X-w[0])*cos) - ((Y-h[0])*sin) + w[0];
                y[i] = ((X-w[0])*sin) + ((Y-h[0])*cos) + h[0]; 
                x_rot[i] = x[i];
                y_rot[i] = y[i];
                }

            console.log("inside data rotation, after rotation");
            console.log(x[0]);
            
            for (const i of renderers){
            var xval= i.data_source.data['x'];
            var yval= i.data_source.data['y'];

            var X=0;
            var Y=0;
            for (var z = 0; z < xval.length; z++){
                  X= xval[z];
                  Y= yval[z];
                  xval[z] = ((X-w[0])*cos) - ((Y-h[0])*sin) + w[0];
                  yval[z] = ((X-w[0])*sin) + ((Y-h[0])*cos) + h[0];}

            i.data_source.change.emit();}

            let x_sum = 0;
            let y_sum = 0;
            for (let i = 0; i < x_rot.length; i++) {
                x_sum += x_rot[i];
                y_sum += y_rot[i];
            }
            
            //w[0] = x_sum/x_rot.length;
            //h[0] = y_sum/y_rot.length;
            //rotation_values.change.emit();
            
            table.change.emit();
            rotval[0] = V;
            
            // for undo
            
            function handleNewWidgetOperation() {
                
                redo_file.data['method'] = [];
                redo_file.data['value'] = [];
                redo_file.change.emit();
            }
            
            if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0] && iden.data['track_flips'][0]===0) {
                
                var last_index_data_rotation = log__file.data['method'].lastIndexOf('data rotation');
            
                if (log__file.data['value'][last_index_data_rotation] != V){
                
                handleNewWidgetOperation();
                
                log__file.data['method'].push('data rotation');
                log__file.data['value'].push(V);
                log__file.change.emit();

                undo_file.data['method'].push('data rotation');
                undo_file.data['value'].push(V);
                undo_file.change.emit();

                }
            
            }
            
            else {
                var last_index_data_rotation = log__file.data['method'].lastIndexOf('data rotation');
            
                if (log__file.data['value'][last_index_data_rotation] != V){
                log__file.data['method'].push('data rotation');
                log__file.data['value'].push(V);
                log__file.change.emit();

                }
            
                iden.data['undo_in_progress'][0] = false; 
                iden.data['track_flips'][0] = 0;
                iden.change.emit();
            }
            
            // for redo
            
            if (iden.data['redo_in_progress'][0]) {
                var last_index_data_rotation = log__file.data['method'].lastIndexOf('data rotation');

                if (log__file.data['value'][last_index_data_rotation] != V){
                log__file.data['method'].push('data rotation');
                log__file.data['value'].push(V);
                log__file.change.emit();
            }
            iden.data['redo_in_progress'][0] = false; 
            iden.data['track_flips'][0] = 0;
            iden.change.emit();
            }
            
            
        """)

        dict_for_data_pixel_size["callback"] = CustomJS(args=dict(renderers=dict_for_renderers[number_of_tab], 
                                            table=list_for_data_table[number_of_tab], 
                                            log__file = log_file[number_of_tab],
                                            size=dict_for_data_pixel_size["sliders"][number_of_tab], 
                                            division_factor_for_resize=list_for_columndatasource_of_data_resize[number_of_tab],
                                            rotation_values=list_for_rotation_values[number_of_tab],
                                            division_factor_for_resize_scale_bar=division_factor_for_resize_scale_bar,
                                            undo_file = undo_dict[number_of_tab],
                                            redo_file = redo_dict[number_of_tab],
                                            iden = identifier[number_of_tab],
                                            scale_bar_px_size = dict_for_scale_bar_pixel_size["sliders"][number_of_tab]),
                            code="""
            const F = size.value;
            var num_fac = division_factor_for_resize.data['x'];
            var den_fac = division_factor_for_resize.data['y'];

            var resize_fac = num_fac[0] / den_fac[0];

            const S = scale_bar_px_size.value;

            var num_fac_scale_bar = division_factor_for_resize_scale_bar.data['x'];
            var den_fac_scale_bar = division_factor_for_resize_scale_bar.data['y'];
            var resize_fac_scale_bar = num_fac_scale_bar[0]/den_fac_scale_bar[0];

            var x = table.source.data['x'];
            var y = table.source.data['y'];

            var x_clean = [];
            var y_clean = [];

            for (var i = 0; i < x.length; i++){
            if (x[i] != 'NaN'){
            x_clean[i]=x[i];
            y_clean[i]=y[i];}}

            var w = rotation_values.data['x'];
            var h = rotation_values.data['y'];
            let x_sum = 0;
            let y_sum = 0;

            var X=0;
            var Y=0;
            for (var i = 0; i < x_clean.length; i++){
                X= x_clean[i];
                Y= y_clean[i];
                x[i] = (X / F) * resize_fac * resize_fac_scale_bar * S;
                y[i] = (Y / F) * resize_fac * resize_fac_scale_bar * S;
                x_sum += x[i];
                y_sum += y[i];
                }        
            table.change.emit();

            for (const i of renderers){
            var xval= i.data_source.data['x'];
            var yval= i.data_source.data['y'];

            for (var z = 0; z < xval.length; z++){
                  X= xval[z];
                  Y= yval[z];
                  xval[z] = (X / F) * resize_fac * resize_fac_scale_bar * S;
                  yval[z] = (Y / F) * resize_fac * resize_fac_scale_bar * S;}

            i.data_source.change.emit();}

            num_fac[0] = F;
            den_fac[0] = 1;

            num_fac_scale_bar[0] = 1;
            den_fac_scale_bar[0] = S;
            
            w[0] = x_sum/x_clean.length;
            h[0] = y_sum/y_clean.length;
            rotation_values.change.emit();
            
            division_factor_for_resize.change.emit();
            division_factor_for_resize_scale_bar.change.emit();
            
            function handleNewWidgetOperation() {
                
                redo_file.data['method'] = [];
                redo_file.data['value'] = [];
                redo_file.change.emit();
            }
            
            if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
                
                var last_index_data_px_size = log__file.data['method'].lastIndexOf('data pixel size');
            
                if (log__file.data['value'][last_index_data_px_size] != F){
                
                handleNewWidgetOperation();
                
                log__file.data['method'].push('data pixel size');
                log__file.data['value'].push(F);
                log__file.change.emit();

                undo_file.data['method'].push('data pixel size');
                undo_file.data['value'].push(F);
                undo_file.change.emit();

                }
            }
            
            else {
                
                var last_index_data_px_size = log__file.data['method'].lastIndexOf('data pixel size');
            
                if (log__file.data['value'][last_index_data_px_size] != F){
                log__file.data['method'].push('data pixel size');
                log__file.data['value'].push(F);
                log__file.change.emit();

                }
            
                iden.data['undo_in_progress'][0] = false; 
                iden.change.emit();
            }
            
            if (iden.data['redo_in_progress'][0]) {
                var last_index_data_px_size = log__file.data['method'].lastIndexOf('data pixel size');
            
                if (log__file.data['value'][last_index_data_px_size] != F){
                log__file.data['method'].push('data pixel size');
                log__file.data['value'].push(F);
                log__file.change.emit();

                }
            iden.data['redo_in_progress'][0] = false; 
            iden.change.emit();
            }
            
        """)

        dict_for_he_rot_angle["callback"] = CustomJS(args=dict(undo_file = undo_dict[number_of_tab], 
                                                               log__file = log_file[number_of_tab], 
                                                               im = im, 
                                                               redo_file = redo_dict[number_of_tab],
                                                               iden = identifier[number_of_tab],
                                                               rotval_for_img=dict_for_he_rot_angle["sliders"][number_of_tab]),
                            code="""
                            const radians = (Math.PI / 180) * rotval_for_img.value;
                            im.glyph.angle = radians;  
                            im.change.emit();
                            
                            function handleNewWidgetOperation() {
                
                                redo_file.data['method'] = [];
                                redo_file.data['value'] = [];
                                redo_file.change.emit();
                            }

                            if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
                                
                                var last_index_img_rotation = log__file.data['method'].lastIndexOf('image rotation');
                            
                                if (log__file.data['value'][last_index_img_rotation] != rotval_for_img.value){
                                
                                handleNewWidgetOperation();
                                
                                log__file.data['method'].push('image rotation');
                                log__file.data['value'].push(rotval_for_img.value);
                                log__file.change.emit();

                                undo_file.data['method'].push('image rotation');
                                undo_file.data['value'].push(rotval_for_img.value);
                                undo_file.change.emit();

                                }
                            
                            }
                            
                            else {
                                
                                var last_index_img_rotation = log__file.data['method'].lastIndexOf('image rotation');
                            
                                if (log__file.data['value'][last_index_img_rotation] != rotval_for_img.value){
                                log__file.data['method'].push('image rotation');
                                log__file.data['value'].push(rotval_for_img.value);
                                log__file.change.emit();

                                }
                            
                                iden.data['undo_in_progress'][0] = false; 
                                iden.change.emit();
                            }
                            
                            if (iden.data['redo_in_progress'][0]) {
                                var last_index_img_rotation = log__file.data['method'].lastIndexOf('image rotation');
                            
                                if (log__file.data['value'][last_index_img_rotation] != rotval_for_img.value){
                                log__file.data['method'].push('image rotation');
                                log__file.data['value'].push(rotval_for_img.value);
                                log__file.change.emit();

                                }
                            iden.data['redo_in_progress'][0] = false; 
                            iden.change.emit();
                            }

            
        """)
        
        dict_for_data_rot_angle["sliders"][number_of_tab].js_on_change('value', dict_for_data_rot_angle["callback"])
        dict_for_data_rot_angle["sliders"][number_of_tab].js_on_change('value', dict_for_data_pixel_size["callback"])
        
        dict_for_data_pixel_size["sliders"][number_of_tab].js_on_change('value', dict_for_data_pixel_size["callback"])
        dict_for_he_rot_angle["sliders"][number_of_tab].js_on_change('value', dict_for_he_rot_angle["callback"])

        dict_for_data_rot_angle["spinners"][number_of_tab].js_link('value', dict_for_data_rot_angle["sliders"][number_of_tab], 'value')
        dict_for_data_rot_angle["sliders"][number_of_tab].js_link('value', dict_for_data_rot_angle["spinners"][number_of_tab], 'value')
        
        dict_for_data_pixel_size["spinners"][number_of_tab].js_link('value', dict_for_data_pixel_size["sliders"][number_of_tab], 'value')
        dict_for_data_pixel_size["sliders"][number_of_tab].js_link('value', dict_for_data_pixel_size["spinners"][number_of_tab], 'value')
        
        dict_for_he_rot_angle["spinners"][number_of_tab].js_link('value', dict_for_he_rot_angle["sliders"][number_of_tab], 'value')
        dict_for_he_rot_angle["sliders"][number_of_tab].js_link('value', dict_for_he_rot_angle["spinners"][number_of_tab], 'value')

        dict_for_toggle_vert["toggle_buttons"][number_of_tab].js_on_click(CustomJS(args=dict(log__file = log_file[number_of_tab], 
                                      rotation_values=list_for_rotation_values[number_of_tab],
                                      renderers=dict_for_renderers[number_of_tab], 
                                        table=list_for_data_table[number_of_tab],
                                    rot=dict_for_data_rot_angle["sliders"][number_of_tab],
                                    vert_toggle=dict_for_toggle_vert["toggle_buttons"][number_of_tab],
                                    undo_file = undo_dict[number_of_tab],
                                    iden = identifier[number_of_tab],
                                    redo_file = redo_dict[number_of_tab],
                                    toggle_button=dict_for_toggle_vert["toggle_buttons"][number_of_tab]), code="""

            console.log("inside verti toggle");
            console.log(toggle_button.active);
            
            var x = table.source.data['x'];
            var y = table.source.data['y'];
            
            var rotval = rotation_values.data['rotval'];

            const V = 0;
            const A = V - rotval[0];

            // rotation angle 0 degrees

            console.log("inside flip, before zero degree/reverse rotation");
            console.log(x[0]);
            
            const radians = (Math.PI / 180) * A;
            const cos = Math.cos(radians);
            const sin = Math.sin(radians);

            var x_clean = [];
            var y_clean = [];

            for (var i = 0; i < x.length; i++){
            if (x[i] != 'NaN'){
            x_clean[i]=x[i];
            y_clean[i]=y[i]; }}

            var w = rotation_values.data['x'];
            var h = rotation_values.data['y'];

            var X=0;
            var Y=0;
            var x_rot = [];
            var y_rot = [];
            
            for (var i = 0; i < x_clean.length; i++) {
                X= x_clean[i];
                Y= y_clean[i];
                x[i] = ((X-w[0])*cos) - ((Y-h[0])*sin) + w[0];
                y[i] = ((X-w[0])*sin) + ((Y-h[0])*cos) + h[0]; 
                x_rot[i] = x[i];
                y_rot[i] = y[i];
                }

            console.log("inside flip, after zero degree rotation");
            console.log(x[0]);
            
            for (const i of renderers){
            var xval= i.data_source.data['x'];
            var yval= i.data_source.data['y'];

            var X=0;
            var Y=0;
            for (var z = 0; z < xval.length; z++){
                  X= xval[z];
                  Y= yval[z];
                  xval[z] = ((X-w[0])*cos) - ((Y-h[0])*sin) + w[0];
                  yval[z] = ((X-w[0])*sin) + ((Y-h[0])*cos) + h[0];}

            i.data_source.change.emit();}

            console.log("before flip operation");

            let x_sum = 0;
            let y_sum = 0;
            for (let i = 0; i < x_rot.length; i++) {
                x_sum += x_rot[i];
                y_sum += y_rot[i];
            }
            
            var x_clean = [];
            var y_clean = [];

            for (var i = 0; i < x.length; i++){
            if (x[i] != 'NaN'){
            x_clean[i]=x[i];
            y_clean[i]=y[i];
            }}
            
            var X=0;
            var Y=0;
            
            for (var i = 0; i < x_clean.length; i++){
            
            X = x_clean[i];
            Y = y_clean[i];
            
            x[i] = w[0] + (w[0] - x[i]); 
            
            }

            console.log("after flip operation");
            console.log(x[0]);

            for (const i of renderers){
            
            var xval= i.data_source.data['x'];
            var yval= i.data_source.data['y'];
            
            for (var z = 0; z < xval.length; z++){
                  
                  X = xval[z];
                  Y = yval[z];

                  xval[z] = w[0] + (w[0] - xval[z]);

            }
            i.data_source.change.emit();
            }
            
            table.change.emit();

            rotval[0] = 0;
            
            //var neg_rotation = rot.value;
            //var A = -neg_rotation;

            console.log("rotation A degrees ahead");
            
            rot.value = -rot.value;
            
            function handleNewWidgetOperation() {
                
                redo_file.data['method'] = [];
                redo_file.data['value'] = [];
                redo_file.change.emit();
            }
            
            if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
                
                var last_index_vert_flip = log__file.data['method'].lastIndexOf('vertical flip');
            
                if (log__file.data['value'][last_index_vert_flip] != vert_toggle.active){
                
                handleNewWidgetOperation();
                
                log__file.data['method'].push('vertical flip');
                log__file.data['value'].push(vert_toggle.active);
                log__file.change.emit();

                undo_file.data['method'].push('vertical flip');
                undo_file.data['value'].push(vert_toggle.active);
                undo_file.change.emit();

                }
            }
            
            else {
                
                var last_index_vert_flip = log__file.data['method'].lastIndexOf('vertical flip');
            
                if (log__file.data['value'][last_index_vert_flip] != vert_toggle.active){
                log__file.data['method'].push('vertical flip');
                log__file.data['value'].push(vert_toggle.active);
                log__file.change.emit();
                }
            
                iden.data['undo_in_progress'][0] = false; 
                iden.change.emit();
            }
            
            if (iden.data['redo_in_progress'][0]) {
                var last_index_vert_flip = log__file.data['method'].lastIndexOf('vertical flip');
            
                if (log__file.data['value'][last_index_vert_flip] != vert_toggle.active){
                log__file.data['method'].push('vertical flip');
                log__file.data['value'].push(vert_toggle.active);
                log__file.change.emit();
                }
            iden.data['redo_in_progress'][0] = false; 
            iden.change.emit();
            }
            
        """))

        dict_for_toggle_hori["toggle_buttons"][number_of_tab].js_on_click(CustomJS(args=dict(log__file = log_file[number_of_tab], rotation_values=list_for_rotation_values[number_of_tab],
                                      renderers=dict_for_renderers[number_of_tab], 
                                        table=list_for_data_table[number_of_tab],
                                        iden = identifier[number_of_tab],
                                    rot=dict_for_data_rot_angle["sliders"][number_of_tab],
                                    horiz_toggle=dict_for_toggle_hori["toggle_buttons"][number_of_tab],
                                    undo_file = undo_dict[number_of_tab],
                                    redo_file = redo_dict[number_of_tab],
                                    toggle_button=dict_for_toggle_hori["toggle_buttons"][number_of_tab]), code="""
            
            console.log("inside horiz toggle");
            console.log(toggle_button.active);
            
            var x = table.source.data['x'];
            var y = table.source.data['y'];
            
            var rotval = rotation_values.data['rotval'];

            const V = 0;
            const A = V - rotval[0];

            // rotation angle 0 degrees

            console.log("inside flip, before zero degree/reverse rotation");
            console.log(x[0]);
            
            const radians = (Math.PI / 180) * A;
            const cos = Math.cos(radians);
            const sin = Math.sin(radians);

            var x_clean = [];
            var y_clean = [];

            for (var i = 0; i < x.length; i++){
            if (x[i] != 'NaN'){
            x_clean[i]=x[i];
            y_clean[i]=y[i]; }}

            var w = rotation_values.data['x'];
            var h = rotation_values.data['y'];

            var X=0;
            var Y=0;
            var x_rot = [];
            var y_rot = [];
            
            for (var i = 0; i < x_clean.length; i++) {
                X= x_clean[i];
                Y= y_clean[i];
                x[i] = ((X-w[0])*cos) - ((Y-h[0])*sin) + w[0];
                y[i] = ((X-w[0])*sin) + ((Y-h[0])*cos) + h[0]; 
                x_rot[i] = x[i];
                y_rot[i] = y[i];
                }

            console.log("inside flip, after zero degree rotation");
            console.log(x[0]);
            
            for (const i of renderers){
            var xval= i.data_source.data['x'];
            var yval= i.data_source.data['y'];

            var X=0;
            var Y=0;
            for (var z = 0; z < xval.length; z++){
                  X= xval[z];
                  Y= yval[z];
                  xval[z] = ((X-w[0])*cos) - ((Y-h[0])*sin) + w[0];
                  yval[z] = ((X-w[0])*sin) + ((Y-h[0])*cos) + h[0];}

            i.data_source.change.emit();}

            console.log("before flip operation");

            var x_clean = [];
            var y_clean = [];

            for (var i = 0; i < x.length; i++){
            if (x[i] != 'NaN'){
            x_clean[i]=x[i];
            y_clean[i]=y[i];
            }}
            
            var X=0;
            var Y=0;
            
            for (var i = 0; i < x_clean.length; i++){
            
            X = x_clean[i];
            Y = y_clean[i];
             
            y[i] = h[0] + (h[0] - y[i]); 
            
            }
            
            for (const i of renderers){
            
            var xval= i.data_source.data['x'];
            var yval= i.data_source.data['y'];
            
            for (var z = 0; z < xval.length; z++){
                  
                  X = xval[z];
                  Y = yval[z];
        
                  yval[z] = h[0] + (h[0] - yval[z]);

            }
            i.data_source.change.emit();
            }
            
            table.change.emit();

            rotval[0] = 0;
            
            //var neg_rotation = rot.value;
            //var A = -neg_rotation;

            console.log("rotation A degrees ahead");
            
            rot.value = -rot.value;
    
            //log__file.data['method'].pop();
            //log__file.data['value'].pop();
            
            function handleNewWidgetOperation() {
                
                redo_file.data['method'] = [];
                redo_file.data['value'] = [];
                redo_file.change.emit();
            }
            
            if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
                
                var last_index_horiz_flip = log__file.data['method'].lastIndexOf('horizontal flip');
            
                if (log__file.data['value'][last_index_horiz_flip] != horiz_toggle.active){
                
                handleNewWidgetOperation();
                
                log__file.data['method'].push('horizontal flip');
                log__file.data['value'].push(horiz_toggle.active);
                log__file.change.emit();

                undo_file.data['method'].push('horizontal flip');
                undo_file.data['value'].push(horiz_toggle.active);
                undo_file.change.emit();

                }
            }
            
            else {
                
                var last_index_horiz_flip = log__file.data['method'].lastIndexOf('horizontal flip');
            
                if (log__file.data['value'][last_index_horiz_flip] != horiz_toggle.active){
                log__file.data['method'].push('horizontal flip');
                log__file.data['value'].push(horiz_toggle.active);
                log__file.change.emit();

                }
                iden.data['undo_in_progress'][0] = false; 
                iden.change.emit();
            }
            
            if (iden.data['redo_in_progress'][0]) {
                var last_index_horiz_flip = log__file.data['method'].lastIndexOf('horizontal flip');
            
                if (log__file.data['value'][last_index_horiz_flip] != horiz_toggle.active){
                log__file.data['method'].push('horizontal flip');
                log__file.data['value'].push(horiz_toggle.active);
                log__file.change.emit();

                }
            iden.data['redo_in_progress'][0] = false; 
            iden.change.emit();
            }
            
        """))
        
        
        dict_for_data_translocation_X["callback"] = CustomJS(args=dict(log__file = log_file[number_of_tab],
                                            renderers=dict_for_renderers[number_of_tab], 
                                            table=list_for_data_table[number_of_tab], 
                                            iden = identifier[number_of_tab],
                                            rotation_values=list_for_rotation_values[number_of_tab],
                                            X_coord = dict_for_data_translocation_X["spinners"][number_of_tab],
                                X_coord_values_for_translocation = list_for_translocation_values[number_of_tab],
                                undo_file = undo_dict[number_of_tab],
                                redo_file = redo_dict[number_of_tab]),
        
        code="""          
                            
        var x = table.source.data['x'];
        var y = table.source.data['y'];
        
        var for_adding_X_coord_values_for_translocation = X_coord_values_for_translocation.data['x'];
        
        const translocation_val = X_coord.value;
            
        var X=0;
        var Y=0;
        
        var x_clean = [];
        var y_clean = [];

        for (var i = 0; i < x.length; i++){
        if (x[i] != 'NaN'){
        x_clean[i]=x[i];
        y_clean[i]=y[i];
        }}

        let x_sum = 0;
        let y_sum = 0;
            
        for (var i = 0; i < x.length; i++){
            X = x[i];
            x[i] = X + translocation_val - for_adding_X_coord_values_for_translocation[0];
            x_sum += x[i];
            y_sum += y[i];
            }
            
        table.change.emit();

        for (const i of renderers){
        var xval= i.data_source.data['x'];

        for (var z = 0; z < xval.length; z++){
              X = xval[z];
              xval[z] = X + translocation_val - for_adding_X_coord_values_for_translocation[0];
            }

        i.data_source.change.emit();}
        
        for_adding_X_coord_values_for_translocation[0] = translocation_val;
        
        var w = rotation_values.data['x'];
        var h = rotation_values.data['y'];
        var rotval = rotation_values.data['rotval'];
        
        w[0] = x_sum/x_clean.length;
        h[0] = y_sum/y_clean.length;
        rotation_values.change.emit();
            
        X_coord_values_for_translocation.change.emit();
        
        function handleNewWidgetOperation() {
                
        redo_file.data['method'] = [];
        redo_file.data['value'] = [];
        redo_file.change.emit();
        }
            
        if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
            
            var last_index_translocation_x = log__file.data['method'].lastIndexOf('translocation in x-axis');

            if (log__file.data['value'][last_index_translocation_x] != translocation_val){
            
            handleNewWidgetOperation();
            
            log__file.data['method'].push('translocation in x-axis');
            log__file.data['value'].push(translocation_val);
            log__file.change.emit();

            undo_file.data['method'].push('translocation in x-axis');
            undo_file.data['value'].push(translocation_val);
            undo_file.change.emit();

            }
        }
        
        else {
            var last_index_translocation_x = log__file.data['method'].lastIndexOf('translocation in x-axis');
        
            if (log__file.data['value'][last_index_translocation_x] != translocation_val){
            log__file.data['method'].push('translocation in x-axis');
            log__file.data['value'].push(translocation_val);
            log__file.change.emit();

            }
        
            iden.data['undo_in_progress'][0] = false; 
            iden.change.emit();
        }
        
        if (iden.data['redo_in_progress'][0]) {
            var last_index_translocation_x = log__file.data['method'].lastIndexOf('translocation in x-axis');
        
            if (log__file.data['value'][last_index_translocation_x] != translocation_val){
            log__file.data['method'].push('translocation in x-axis');
            log__file.data['value'].push(translocation_val);
            log__file.change.emit();

            }
        iden.data['redo_in_progress'][0] = false; 
        iden.change.emit();
        }
        
        """)
        
        dict_for_data_translocation_X["spinners"][number_of_tab].js_on_change('value',  dict_for_data_translocation_X["callback"])
         
        dict_for_data_translocation_Y["callback"] = CustomJS(args=dict(log__file = log_file[number_of_tab], 
                                            renderers=dict_for_renderers[number_of_tab], 
                                            table=list_for_data_table[number_of_tab], 
                                            iden = identifier[number_of_tab],
                                            rotation_values=list_for_rotation_values[number_of_tab],
                                            Y_coord = dict_for_data_translocation_Y["spinners"][number_of_tab],
                                   Y_coord_values_for_translocation = list_for_translocation_values[number_of_tab],
                                   undo_file = undo_dict[number_of_tab],
                                   redo_file = redo_dict[number_of_tab]),
        
        code="""          
                            
        var y = table.source.data['y'];
        var x = table.source.data['x'];
        
        var for_adding_Y_coord_values_for_translocation = Y_coord_values_for_translocation.data['y'];
        const translocation_val = Y_coord.value;

        var Y=0;
        var X=0;
        
        var x_clean = [];
        var y_clean = [];

        for (var i = 0; i < x.length; i++){
        if (x[i] != 'NaN'){
        x_clean[i]=x[i];
        y_clean[i]=y[i];
        }}

        let x_sum = 0;
        let y_sum = 0;
        
        for (var i = 0; i < y.length; i++){
            Y=y[i];
            y[i] = Y + translocation_val - for_adding_Y_coord_values_for_translocation[0];
            x_sum += x[i];
            y_sum += y[i];
            }
            
        table.change.emit();


        for (const i of renderers){
        var yval= i.data_source.data['y'];

        for (var z = 0; z < yval.length; z++){
              Y= yval[z];
              yval[z] = Y + translocation_val - for_adding_Y_coord_values_for_translocation[0];
            }

        i.data_source.change.emit();}
        
        var w = rotation_values.data['x'];
        var h = rotation_values.data['y'];
        var rotval = rotation_values.data['rotval'];
        
        w[0] = x_sum/x_clean.length;
        h[0] = y_sum/y_clean.length;
        rotation_values.change.emit();
        
        for_adding_Y_coord_values_for_translocation[0] = translocation_val;
        Y_coord_values_for_translocation.change.emit();
        
        function handleNewWidgetOperation() {
                
            redo_file.data['method'] = [];
            redo_file.data['value'] = [];
            redo_file.change.emit();
        }

        if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
            
            var last_index_translocation_y = log__file.data['method'].lastIndexOf('translocation in y-axis');
        
            if (log__file.data['value'][last_index_translocation_y] != translocation_val){
            
            handleNewWidgetOperation();
            
            log__file.data['method'].push('translocation in y-axis');
            log__file.data['value'].push(translocation_val);
            log__file.change.emit();

            undo_file.data['method'].push('translocation in y-axis');
            undo_file.data['value'].push(translocation_val);
            undo_file.change.emit();

            }
        }
        
        else {
            var last_index_translocation_y = log__file.data['method'].lastIndexOf('translocation in y-axis');
        
            if (log__file.data['value'][last_index_translocation_y] != translocation_val){
            log__file.data['method'].push('translocation in y-axis');
            log__file.data['value'].push(translocation_val);
            log__file.change.emit();

            }
            iden.data['undo_in_progress'][0] = false; 
            iden.change.emit();
        }
        
        if (iden.data['redo_in_progress'][0]) {
            var last_index_translocation_y = log__file.data['method'].lastIndexOf('translocation in y-axis');
        
            if (log__file.data['value'][last_index_translocation_y] != translocation_val){
            log__file.data['method'].push('translocation in y-axis');
            log__file.data['value'].push(translocation_val);
            log__file.change.emit();

            }
        iden.data['redo_in_progress'][0] = false; 
        iden.change.emit();
        }
        
        """)
        
        dict_for_data_translocation_Y["spinners"][number_of_tab].js_on_change('value',  dict_for_data_translocation_Y["callback"])

        callback_for_scale_bar = CustomJS(args=dict(log__file = log_file[number_of_tab], my_arrow=my_arrow, labels=labels, p=p,
                                    scale_bar_px_size = dict_for_scale_bar_pixel_size["sliders"][number_of_tab],
                                    undo_file = undo_dict[number_of_tab],
                                    iden = identifier[number_of_tab],
                                    redo_file = redo_dict[number_of_tab]),
                                 code="""

                const Size = scale_bar_px_size.value;
                const S = Size * 1000; // 1000 for 1000 pixels

                my_arrow.x_end = p.x_range.end - 500;
                my_arrow.x_start = p.x_range.end - (500+S);
                my_arrow.y_end = p.y_range.start + 600;
                my_arrow.y_start = p.y_range.start + 600;
                my_arrow.visible = true;

                labels.x = p.x_range.end - (500+S);
                labels.y = p.y_range.start + 600;
                labels.text = `${Size} mm`;
                labels.x_offset = 15 * Size;
                labels.visible = true;

                my_arrow.change.emit();
                labels.change.emit();
                
                function handleNewWidgetOperation() {
                
                    redo_file.data['method'] = [];
                    redo_file.data['value'] = [];
                    redo_file.change.emit();
                }

                if (!iden.data['undo_in_progress'][0] && !iden.data['redo_in_progress'][0]) {
                    
                    var last_index_scale_px_size = log__file.data['method'].lastIndexOf('scale bar pixel size');

                    if (log__file.data['value'][last_index_scale_px_size] != Size){

                    handleNewWidgetOperation();
                    
                    log__file.data['method'].push('scale bar pixel size');
                    log__file.data['value'].push(Size);
                    log__file.change.emit();

                    undo_file.data['method'].push('scale bar pixel size');
                    undo_file.data['value'].push(Size);
                    undo_file.change.emit();

                    }
                
                }
                
                else {
                    
                    var last_index_scale_px_size = log__file.data['method'].lastIndexOf('scale bar pixel size');

                    if (log__file.data['value'][last_index_scale_px_size] != Size){
                    log__file.data['method'].push('scale bar pixel size');
                    log__file.data['value'].push(Size);
                    log__file.change.emit();

                    }
                    iden.data['undo_in_progress'][0] = false; 
                    iden.change.emit();
                }
                
                if (iden.data['redo_in_progress'][0]) {
                    var last_index_scale_px_size = log__file.data['method'].lastIndexOf('scale bar pixel size');

                    if (log__file.data['value'][last_index_scale_px_size] != Size){
                    log__file.data['method'].push('scale bar pixel size');
                    log__file.data['value'].push(Size);
                    log__file.change.emit();

                    }
                iden.data['redo_in_progress'][0] = false; 
                iden.change.emit();
                }
                
                """)

        #p.js_on_event('pan', dict_for_multichoice["callback"])
        #p.js_on_event('pan', dict_for_data_pixel_size["callback"])
        #p.js_on_event('mouseenter', dict_for_data_pixel_size["callback"])

        dict_for_scale_bar_pixel_size["sliders"][number_of_tab].js_on_change('value',  callback_for_scale_bar)
        p.js_on_event(RangesUpdate, callback_for_scale_bar)
        dict_for_scale_bar_pixel_size["spinners"][number_of_tab].js_link('value', dict_for_scale_bar_pixel_size["sliders"][number_of_tab], 'value')
        dict_for_scale_bar_pixel_size["sliders"][number_of_tab].js_link('value', dict_for_scale_bar_pixel_size["spinners"][number_of_tab], 'value')
        dict_for_scale_bar_pixel_size["sliders"][number_of_tab].js_on_change('value', dict_for_img_number["callback"])
        dict_for_scale_bar_pixel_size["sliders"][number_of_tab].js_on_change('value', dict_for_data_pixel_size["callback"])

        div_orientation.append(Div(text="""Change Orientation of Data:""", visible = visibility, height=15))
        div_translocation.append(Div(text="""Translocation of Data:""", visible = visibility, width=1000, height=15))
        div_table_heading.append(Div(text="""Updated Spatial Coordinates (x, y): """, visible = visibility, height=15))
        div_preview_imgs.append(Div(text="""Preview of Images: """, visible = visibility, height=15))
        div_px_size.append(Div(text="""Pixel Size (in um/px) Range: 0.01 to 3 =""", visible = visibility, height=15))
        div_rot_angle.append(Div(text="""Rotation Angle (in degrees) Range: -360 to 360 =""", visible = visibility, height=15))
        div_download_heading.append(Div(text="""Download:""", visible = visibility, height=15))

        div_undo_redo.append(Div(text="""Undo-Redo:""", visible = visibility, height=15))

        div4space1.append(Div(text="""""", visible = visibility))
        div4space2.append(Div(text="""""", visible = visibility))
        div4space3.append(Div(text="""""", visible = visibility))
        div4space4.append(Div(text="""""", visible = visibility, height=10))
        div4space5.append(Div(text="""""", visible = visibility, height=10))
        div4space6.append(Div(text="""""", visible = visibility))
        div4space7.append(Div(text="""""", visible = visibility))

        non_linear_callback_and_for_download = CustomJS(args=dict(combined_source = combined_source[number_of_tab],
                      image_number = dict_for_img_number["spinners"][number_of_tab], 
                      non_linear_button = list_of_non_linear_buttons[number_of_tab],
                      linear_button = list_of_linear_buttons[number_of_tab],
                      img_px_size = dict_for_he_pixel_size["spinners"][number_of_tab],                      
                     angle=dict_for_data_rot_angle["spinners"][number_of_tab],
                      img_rotation = dict_for_he_rot_angle["spinners"][number_of_tab],
                    px_size=dict_for_data_pixel_size["spinners"][number_of_tab],
                     display_scale=dict_for_scale_bar_pixel_size["spinners"][number_of_tab],
                     hori_toggle = dict_for_toggle_hori["toggle_buttons"][number_of_tab],
                     vert_toggle = dict_for_toggle_vert["toggle_buttons"][number_of_tab],
                    translocate_X = dict_for_data_translocation_X["spinners"][number_of_tab],
                      translocate_Y = dict_for_data_translocation_Y["spinners"][number_of_tab],
                    scale_bar_px_size = dict_for_scale_bar_pixel_size["spinners"][number_of_tab],
                      renderers=dict_for_renderers[number_of_tab],
                     data_point_size = spinners_for_all_data_point_size[number_of_tab],
                     data_point_size_non_linear = spinners_for_all_data_point_size_non_linear[number_of_tab],
                     log__file = log_file[number_of_tab], 
                      div_px_size = div_px_size[number_of_tab],
                      div4space1 = div4space1[number_of_tab], div4space2 = div4space2[number_of_tab],
                      div_rot_angle = div_rot_angle[number_of_tab], 
                      div4space3 = div4space3[number_of_tab],
                      div4space4 = div4space4[number_of_tab],
                      div4space5 = div4space5[number_of_tab],
                      div4space6 = div4space6[number_of_tab],
                      div4space7 = div4space7[number_of_tab],
                      div_orientation = div_orientation[number_of_tab],
                      redo_buttons = dict_redo_button['buttons'][number_of_tab],
                      undo_buttons = dict_undo_button['buttons'][number_of_tab],
                    threshold=dict_for_threshold["spinners"][number_of_tab],
                    div_translocation = div_translocation[number_of_tab],
                    renderers_non_linear=dict_for_renderers_non_linear[number_of_tab],
                    table_linear=list_for_data_table[number_of_tab],
                    draw_tool=draw_tool[number_of_tab],
                    legend_plot=legend_plot[number_of_tab],
                    div_cluster_heading = div_cluster_heading[number_of_tab],
                    div_undo_redo=div_undo_redo[number_of_tab],
                    div_download_heading=div_download_heading[number_of_tab],
                    div_preview_imgs=div_preview_imgs[number_of_tab],
                    divs_of_imgs=divs_of_imgs,
                    div_table_heading=div_table_heading[number_of_tab]), code="""

              non_linear_button.button_type = 'primary';
              
              for (var i = 0; i < renderers.length; i++){
                    //renderers[i].glyph.fill_alpha=0;
                    //renderers[i].glyph.line_alpha=0;

                    renderers[i].visible=false;
                }

              for (var i = 0; i < renderers_non_linear.length; i++){
                    renderers_non_linear[i].visible=true;
                }

                draw_tool.active=true;
                
                vert_toggle.visible=false;
                hori_toggle.visible=false;
                image_number.visible=false;
                px_size.visible=false;
                angle.visible=false;
                img_px_size.visible=false;
                img_rotation.visible=false;
                div_px_size.visible=false;
                div4space1.visible=false;
                div4space2.visible=false;
                div_rot_angle.visible=false;
                div4space3.visible=false;
                div4space4.visible=false;
                div4space5.visible=false;
                div4space6.visible=false;
                div_orientation.visible=false;
                scale_bar_px_size.visible=false;
                translocate_X.visible=false;
                translocate_Y.visible=false;
                div_translocation.visible=false;
                redo_buttons.visible=false;
                undo_buttons.visible=false;
                div4space7.visible=false;
                table_linear.visible=false;
                data_point_size.visible=false;
                div_cluster_heading.visible=false;
                div_undo_redo.visible=false;
                //div_preview_imgs.visible=false;
                div_table_heading.visible=false;
                
                //for (var i = 0; i < divs_of_imgs.length; i++){
                //    divs_of_imgs[i].visible=false;
                //}

                legend_plot.visible=false;

                var legend_plot_legend = legend_plot.renderers;

                for (var i = 0; i < legend_plot_legend.length; i++){
                    legend_plot.renderers[i].visible=false;
                }

                threshold.visible=true;
                data_point_size_non_linear.visible=true;

                var remaining_data_X = combined_source.data['x'];
                var remaining_data_Y = combined_source.data['y'];
                var remaining_data_cell_ids = combined_source.data['CellIDs'];
                
                var remaining_data_X_INITIAL = combined_source.data['original_x'];
                var remaining_data_Y_INITIAL = combined_source.data['original_y'];
                
                const A = angle.value;
                const radians = (Math.PI / 180) * A;
                
                const F = px_size.value;
                const S = display_scale.value;
                const P_img = img_px_size.value;
                const img_rot = img_rotation.value;
                
                const HT = hori_toggle.active;
                const VT = vert_toggle.active;

                const cos = Math.cos(radians);
                const sin = Math.sin(radians);
                
                let sum_X = 0;
                let sum_Y = 0;
                
                remaining_data_X_INITIAL.forEach( num => {
                  sum_X += num;
                })
                
                remaining_data_Y_INITIAL.forEach( num => {
                  sum_Y += num;
                })
                
                var w = sum_X/remaining_data_X_INITIAL.length;
                var h = sum_Y/remaining_data_Y_INITIAL.length;
                
                const actual_translocation_value_X = translocate_X.value;    
                const actual_translocation_value_Y = translocate_Y.value;
                
                var X=0;
                var Y=0;

                // FLIP
                
                if (VT == true){
                
                var X=0;
                var Y=0;

                for (var i = 0; i < remaining_data_X.length; i++){

                X = remaining_data_X[i];
                Y = remaining_data_Y[i];

                remaining_data_X[i] = w + (w - remaining_data_X[i]); 

                }}
                
                if (HT == true){

                var X=0;
                var Y=0;

                for (var i = 0; i < remaining_data_X.length; i++){

                X = remaining_data_X[i];
                Y = remaining_data_Y[i];

                remaining_data_Y[i] = h + (h - remaining_data_Y[i]); 

                }}
                
                var x_sum=0;
                var y_sum=0;
                
                // SCALE
                
                for (var i = 0; i < remaining_data_X.length; i++){
                    remaining_data_X[i] = (remaining_data_X[i] / F) * S;
                    remaining_data_Y[i] = (remaining_data_Y[i] / F) * S;
                    x_sum = x_sum + remaining_data_X[i];
                    y_sum = y_sum + remaining_data_Y[i];
                    }
                    
                w = x_sum/remaining_data_X.length;
                h = y_sum/remaining_data_Y.length;
                
                // ROTATION
                
                for (var i = 0; i < remaining_data_X.length; i++){
                    X = remaining_data_X[i];
                    Y = remaining_data_Y[i];
                    remaining_data_X[i] = ((X-w)*cos) - ((Y-h)*sin) + w;
                    remaining_data_Y[i] = ((X-w)*sin) + ((Y-h)*cos) + h;}
                
                var x_sum=0;
                var y_sum=0;  
                
                // TRANSLOCATION
                
                for (var i = 0; i < remaining_data_X.length; i++){ 
                    remaining_data_X[i] = remaining_data_X[i] + actual_translocation_value_X;
                    remaining_data_Y[i] = remaining_data_Y[i] + actual_translocation_value_Y;
                    x_sum = x_sum + remaining_data_X[i];
                    y_sum = y_sum + remaining_data_Y[i];
                    }

                combined_source.change.emit();
                non_linear_button.disabled = true;
                linear_button.disabled = true;
    
        """)
        
        dict_for_data_save_button["save_buttons"][number_of_tab].js_on_event(ButtonClick, non_linear_callback_and_for_download)
        list_of_non_linear_buttons[number_of_tab].js_on_event(ButtonClick, non_linear_callback_and_for_download)

        dict_for_data_save_button["save_buttons"][number_of_tab].js_on_event(ButtonClick, CustomJS(
            args=dict(combined_source = combined_source[number_of_tab],
                      image_number = dict_for_img_number["sliders"][number_of_tab], 
                      save_button=dict_for_data_save_button["save_buttons"][number_of_tab],
                      img_px_size = dict_for_he_pixel_size["spinners"][number_of_tab],                      
                     angle=dict_for_data_rot_angle["sliders"][number_of_tab],
                      img_rotation = dict_for_he_rot_angle["sliders"][number_of_tab],
                    px_size=dict_for_data_pixel_size["sliders"][number_of_tab],
                     display_scale=dict_for_scale_bar_pixel_size["sliders"][number_of_tab],
                     hori_toggle = dict_for_toggle_hori["toggle_buttons"][number_of_tab],
                     vert_toggle = dict_for_toggle_vert["toggle_buttons"][number_of_tab],
                    translocate_X = dict_for_data_translocation_X["spinners"][number_of_tab],
                      translocate_Y = dict_for_data_translocation_Y["spinners"][number_of_tab],
                      renderers=dict_for_renderers[number_of_tab],
                     all_clusters = list_for_storing_total_clusters_info_in_each_file[number_of_tab],
                     log__file = log_file[number_of_tab]),
            code="""

                // for saving a .csv file containing updated x-y coordinates          
                
                const remaining_data_X = combined_source.data['x'];
                const remaining_data_Y = combined_source.data['y'];
                const remaining_data_cell_ids = combined_source.data['CellIDs'];

                const remaining_data_X_INITIAL = combined_source.data['original_x'];
                const remaining_data_Y_INITIAL = combined_source.data['original_y'];
            
                var out = "cell id,initial x,initial y,final x,final y\\n";
                
                for (var i = 0; i < remaining_data_X.length; i++) {
                if (remaining_data_X[i] != 'NaN'){
                    out += remaining_data_cell_ids[i] + "," + remaining_data_X_INITIAL[i] + "," + remaining_data_Y_INITIAL[i] + "," + remaining_data_X[i] + "," + remaining_data_Y[i] + "\\n";
                }}
                                
                var file = new Blob([out], {type: 'text/plain'});
                var elem = window.document.createElement('a');
                elem.href = window.URL.createObjectURL(file);
                elem.download = 'Updated_Spatial_Data.csv';
                document.body.appendChild(elem);
                elem.click();
                document.body.removeChild(elem);   
                
                // for saving a .txt file containing widget values for image download
                
                const N = image_number.value;
                
                var x = ['\\n', 'Manipulations for Image:', '\\n', `Pixel Size (in um/px): ${img_px_size.value}`, `Rotation Angle (in degrees): ${img_rotation.value}`, `Index of Chosen Image: ${image_number.value}`, '\\n', 'Manipulations for Data:', '\\n', `Flip around Vertical Plane: ${vert_toggle.active}`, `Flip around Horizontal Plane: ${hori_toggle.active}`, `Pixel Size (in um/px): ${px_size.value}`, `Rotation Angle (in degrees): ${angle.value}`, `Translocation in the x-axis (in px): ${translocate_X.value}`, `Translocation in the y-axis (in px): ${translocate_Y.value}`, '\\n', 'Miscellaneous:', '\\n', `Display Scale (in um/px): ${display_scale.value}`];
                    
                var out = "Widget values for Updated Data and Image";

                for (var i = 0; i < x.length; i++) {
                    out += x[i] + "\\n";
                }
                var file = new Blob([out], {type: 'text/plain'});
                var elem = window.document.createElement('a');
                elem.href = window.URL.createObjectURL(file);                
                elem.download = 'widget_values_for_updated_data_and_image.txt';
                document.body.appendChild(elem);
                elem.click();
                document.body.removeChild(elem);
                
                // for saving a .csv file containing all record logs
                
                var out = "method,value \\n";
                
                for (var i = 0; i < log__file.data['method'].length; i++) {
                    out += log__file.data['method'][i] + "," + log__file.data['value'][i] + "\\n";
                }
                
                var file = new Blob([out], {type: 'text/plain'});
                var elem = window.document.createElement('a');
                elem.href = window.URL.createObjectURL(file);
                elem.download = 'record_log.csv';
                document.body.appendChild(elem);
                elem.click();
                document.body.removeChild(elem);   
                
                // list_of_source_final
                
                """
        ))
                                                       
        tabs.append(TabPanel(child=p, title=title[number_of_tab]))
    
    tabs = Tabs(tabs = tabs)   

    callback_for_tabs = CustomJS(args=dict(tabs = tabs, total_tabs = total_tabs,
                                      vertical_toggle = dict_for_toggle_vert["toggle_buttons"],
                                      horizontal_toggle = dict_for_toggle_hori["toggle_buttons"],
                                      data_px_size = dict_for_data_pixel_size["spinners"],
                                      data_rot_angle = dict_for_data_rot_angle["spinners"],
                                      data_table = list_for_data_table,
                                      save_buttons = dict_for_data_save_button["save_buttons"],
                                      data_point_size = spinners_for_all_data_point_size,
                                      data_point_size_non_linear=spinners_for_all_data_point_size_non_linear,
                                      he_px_size = dict_for_he_pixel_size["spinners"],
                                      he_rot_angle = dict_for_he_rot_angle["spinners"],
                                      he_number = dict_for_img_number["spinners"],
                                      scale_bar_px_size =  dict_for_scale_bar_pixel_size["spinners"],
                                      image_save_buttons = dict_for_image_save_button["save_buttons"],
                                      translocation_X = dict_for_data_translocation_X["spinners"], 
                                      translocation_Y = dict_for_data_translocation_Y["spinners"],
                                      div_cluster_heading = div_cluster_heading, 
                                      div_preview_imgs = div_preview_imgs, 
                                      div_table_heading = div_table_heading, 
                                      div_px_size = div_px_size,
                                      div4space1 = div4space1, div4space2 = div4space2,
                                      div_rot_angle = div_rot_angle, div4space3 = div4space3,
                                      div4space4 = div4space4,
                                      div4space5 = div4space5,
                                      div4space6 = div4space6,
                                      div4space7 = div4space7,
                                      div_orientation = div_orientation,
                                      redo_buttons = dict_redo_button['buttons'],
                                      undo_buttons = dict_undo_button['buttons'],
                                    div_translocation = div_translocation,
                                    list_of_non_linear_buttons = list_of_non_linear_buttons,
                                    list_of_linear_buttons = list_of_linear_buttons,
                                    div_transform_heading =  div_transform_heading,
                                    div_undo_redo=div_undo_redo,
                                    div_download_heading=div_download_heading,
                                    legend_plot=legend_plot,
                                    threshold=dict_for_threshold["spinners"]), code="""

    for (var j=0; j < total_tabs.length; j++){

    vertical_toggle[j].visible=false;
    horizontal_toggle[j].visible=false;
    data_px_size[j].visible=false;
    data_rot_angle[j].visible=false;
    data_table[j].visible=false;
    save_buttons[j].visible=false;
    data_point_size[j].visible=false;
    data_point_size_non_linear[j].visible=false;
    he_px_size[j].visible=false;
    he_rot_angle[j].visible=false;
    he_number[j].visible=false;
    div_cluster_heading[j].visible=false;
    div_preview_imgs[j].visible=false;
    div_table_heading[j].visible=false;
    div_px_size[j].visible=false;
    div4space1[j].visible=false;
    div4space2[j].visible=false;
    div_rot_angle[j].visible=false;
    div4space3[j].visible=false;
    div4space4[j].visible=false;
    div4space5[j].visible=false;
    div4space6[j].visible=false;
    div_orientation[j].visible=false;
    scale_bar_px_size[j].visible=false;
    image_save_buttons[j].visible=false;
    translocation_X[j].visible=false;
    translocation_Y[j].visible=false;
    div_translocation[j].visible=false;
    redo_buttons[j].visible=false;
    undo_buttons[j].visible=false;
    div4space7[j].visible=false;
    list_of_non_linear_buttons[j].visible=false;
    list_of_linear_buttons[j].visible=false;
    div_transform_heading[j].visible=false;
    div_undo_redo[j].visible=false;
    div_download_heading[j].visible=false;
    data_point_size_non_linear[j].visible=false;
    threshold[j].visible=false;
    
    legend_plot[j].visible=true;
    //var legend_plot_legend = legend_plot[j].renderers;

    //for (var i = 0; i < legend_plot_legend.length; i++){
    //    legend_plot_legend[i].visible=true;
    //}
    }

    for (var j=0; j < total_tabs.length; j++){

    if (tabs.active == j){

    vertical_toggle[j].visible=true;
    horizontal_toggle[j].visible=true;
    data_px_size[j].visible=true;
    data_rot_angle[j].visible=true;
    data_table[j].visible=true;
    save_buttons[j].visible=true;
    data_point_size[j].visible=true;    
    he_px_size[j].visible=true;
    he_rot_angle[j].visible=true;
    he_number[j].visible=true;
    div_cluster_heading[j].visible=true;
    div_preview_imgs[j].visible=true;
    div_table_heading[j].visible=true;
    div_px_size[j].visible=true;
    div4space1[j].visible=true;
    div4space2[j].visible=true;
    div_rot_angle[j].visible=true;
    div4space3[j].visible=true;
    div4space4[j].visible=true;
    div4space5[j].visible=true;
    div4space6[j].visible=true;
    div_orientation[j].visible=true;
    scale_bar_px_size[j].visible=true;
    image_save_buttons[j].visible=true;
    translocation_X[j].visible=true;
    translocation_Y[j].visible=true;
    div_translocation[j].visible=true;
    redo_buttons[j].visible=true;
    undo_buttons[j].visible=true;
    div4space7[j].visible=true;
    list_of_non_linear_buttons[j].visible=true;
    list_of_linear_buttons[j].visible=true;
    div_transform_heading[j].visible=true;
    div_undo_redo[j].visible=true;
    div_download_heading[j].visible=true;
    
    legend_plot[j].visible=true;

    //var legend_plot_legend = legend_plot[j].renderers;

    //for (var i = 0; i < legend_plot_legend.length; i++){
    //    legend_plot_legend[i].visible=true;
    //}
    
    data_point_size_non_linear[j].visible=false;
    threshold[j].visible=false;
    }
    if (tabs.active != j) {

    vertical_toggle[j].visible=false;
    horizontal_toggle[j].visible=false;
    data_px_size[j].visible=false;
    data_rot_angle[j].visible=false;
    data_table[j].visible=false;
    save_buttons[j].visible=false;
    data_point_size[j].visible=false;
    he_px_size[j].visible=false;
    he_rot_angle[j].visible=false;
    he_number[j].visible=false;
    div_cluster_heading[j].visible=false;
    div_preview_imgs[j].visible=false;
    div_table_heading[j].visible=false;
    div_px_size[j].visible=false;
    div4space1[j].visible=false;
    div4space2[j].visible=false;
    div_rot_angle[j].visible=false;
    div4space3[j].visible=false;
    div4space4[j].visible=false;
    div4space5[j].visible=false;
    div4space6[j].visible=false;
    div_orientation[j].visible=false;
    scale_bar_px_size[j].visible=false;
    image_save_buttons[j].visible=false;
    translocation_X[j].visible=false;
    translocation_Y[j].visible=false;
    div_translocation[j].visible=false;
    redo_buttons[j].visible=false;
    undo_buttons[j].visible=false;
    div4space7[j].visible=false;
    list_of_non_linear_buttons[j].visible=false;
    list_of_linear_buttons[j].visible=false;
    div_transform_heading[j].visible=false;
    div_undo_redo[j].visible=false;
    div_download_heading[j].visible=false;
    legend_plot[j].visible=false;

    //var legend_plot_legend = legend_plot[j].renderers;

    //for (var i = 0; i < legend_plot_legend.length; i++){
    //    legend_plot_legend[i].visible=false;
    //}

    data_point_size_non_linear[j].visible=false;
    threshold[j].visible=false;
    }
    }

    """)
    
    tabs.js_on_change('active', callback_for_tabs)
        
    # row(dict_for_multichoice["multichoice"]), row(div_orientation), 
    
    show(row(column(tabs, row(div4space1)), column(row(div_transform_heading), row(row(list_of_linear_buttons), row(list_of_non_linear_buttons)),
        row(div_download_heading), row(dict_for_data_save_button["save_buttons"]),
         row(div_cluster_heading), row(legend_plot), 
         row(div_orientation), row(row(dict_for_toggle_vert["toggle_buttons"]), row(dict_for_toggle_hori["toggle_buttons"])),
         row(div_undo_redo), row(row(dict_undo_button['buttons']), row(dict_redo_button['buttons'])),
         row(div4space2), row(spinners_for_all_data_point_size), row(spinners_for_all_data_point_size_non_linear), 
         row(div4space5),       
         row(div_px_size),
         row(row(dict_for_data_pixel_size["spinners"]), row(dict_for_he_pixel_size["spinners"]), row(dict_for_scale_bar_pixel_size["spinners"])), 
         row(div4space4), 
         row(dict_for_threshold["spinners"]),
         row(div_rot_angle),
         row(row(dict_for_data_rot_angle["spinners"]), row(dict_for_he_rot_angle["spinners"])), 
         row(div4space6),
         row(div_translocation), 
         row(row(dict_for_data_translocation_X["spinners"]), row(dict_for_data_translocation_Y["spinners"])),     
         row(div4space7),
         row(dict_for_img_number["spinners"]),
         row(div_preview_imgs), row(divs_of_imgs),
         row(row(div_table_heading), row(div4space3)), 
         row(list_for_data_table))))

In [ ]:
# for saving the modified microscopy image

def image_download(filename):
    path_of_widget_file = filename
    values=[]
    with open(path_of_widget_file, "r") as file:
        for line in file:
            out = re.findall('[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', line)
            if len(out) == 1:
                values.append(float(out[0]))
            
    image = Image.open(imgs[int(values[2])])
    width, height = image.size

    new_width = (width / values[0]) * values[7]
    new_height = (height / values[0]) * values[7]

    image = image.resize((round(new_width), round(new_height)), Image.Resampling.BICUBIC)
    image = image.rotate(values[1], Image.Resampling.NEAREST, expand = 1, fillcolor = (255,255,255))
    image.save("Updated_HE_Image.jpg")

In [ ]:
# for visualization of the updated spatio-molecular data

def plot_of_updated_data(filename):
    changed_coord_plot = pd.read_csv(filename)
    plt.gcf().set_size_inches((40, 40))
    plt.scatter(x=changed_coord_plot['final x'], y=changed_coord_plot['final y'], s=100, alpha=1, marker=".", c='blue')
    plt.xticks(fontsize=30)
    plt.xlabel("x", labelpad=50, fontsize=60)
    plt.yticks(fontsize=30)
    plt.ylabel("y", labelpad=50, fontsize=60)
    plt.title("Updated Spatial Data", fontsize=60, y=1.01)
    plt.show()

In [ ]:
print("All required Modules and Packages have been imported")